## Environment/Import/Configuration Setup

In [ ]:
# Verify Python interpreter the notebook is using (for debug)
import sys
print(sys.executable)

In [ ]:
#Configure Pandas to avoid future warnings
import pandas as pd
pd.set_option('future.no_silent_downcasting', True)

In [ ]:
# Put down some non-critical warnings to keep the notebook output clean
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
# Force Matplotlib to show  plots inside the notebook, below the code 
import matplotlib
matplotlib.use('module://matplotlib_inline.backend_inline')
%matplotlib inline

In [ ]:
# Import necessary packages to see if everything is installed and works
import pandas as pd
import matplotlib.pyplot as plt
from socceraction.data.statsbomb import StatsBombLoader
from socceraction.spadl import statsbomb as spadl
from socceraction.vaep import features, labels
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
tqdm.pandas()

In [ ]:
# Configure Pandas to display all columns and rows
import pandas as pd
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None)     
pd.set_option('display.width', None)        
pd.set_option('display.max_colwidth', None) 

## The 19 playing styles adapted (Best performance)

In [ ]:
# all libraries used in the code
import joblib
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
import plotly.express as px
from socceraction.data.statsbomb import StatsBombLoader
from socceraction.spadl import statsbomb as spadl_sb
from socceraction import spadl
from socceraction.vaep import features as feat
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering, DBSCAN, Birch
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score


#Load StatsBomb data locally
sbl = StatsBombLoader(
    getter="local",  # from local files
    root="C:/Users/helio/MachineLearning/Bachelor/statsbomb_data/open-data-master/data"
)

# Get all games for a given competition and season
# Competition ID = 2, Season ID = 27 (Premier League 2015/16)
# Competition ID = 11, Season ID = 27 (La Liga 2015/16)
# Competition ID = 9, Season ID = 27 (Bundesliga 2015/16)
# Competition ID = 12, Season ID = 27 (Serie A 2015/16)
# Competition ID = 7, Season ID = 27 (Ligue 1 2015/16)
games = sbl.games(2, 27)

# Load pre-trained VAEP model that some metrics need, especially VAEP metrics
data = joblib.load("C:/Users/helio/MachineLearning/Bachelor/vaep_model_xgboost_5leagues_1516.pkl")
# The trained model
model = data["model"]          
# Expected input features for model
expected_columns = data["feature_columns"] 


# Create a mapping of team ids to names for all matches of interest
matches_of_interest = games[["game_id", "home_team_id", "away_team_id"]].copy()
team_id_name_map = {}
for game_id in matches_of_interest["game_id"]:
    teams = sbl.teams(game_id)
    for _, row in teams.iterrows():
        team_id_name_map[row["team_id"]] = row["team_name"]

# Add team names to match dataframe
matches_of_interest["home_team_name"] = matches_of_interest["home_team_id"].map(team_id_name_map)
matches_of_interest["away_team_name"] = matches_of_interest["away_team_id"].map(team_id_name_map)

# Keep only rows with valid team names
matches_of_interest = matches_of_interest.dropna(subset=["home_team_name", "away_team_name"])


# compute team metrics for a single match
def evaluate_team_metrics(game_id, home_name, away_name, model):
    # Load game and prepare base dataframe
    match = games.loc[games["game_id"] == game_id].iloc[0]
    home_id, away_id = match["home_team_id"], match["away_team_id"]
    events = sbl.events(game_id)
    # convert to SPADL actions
    actions = spadl.add_names(spadl_sb.convert_to_actions(events, home_id))
    df = pd.DataFrame(index=[home_id, away_id])

    # Predict VAEP for each action (for VAEP metrics only)
    gamestates = feat.play_left_to_right(feat.gamestates(actions, nb_prev_actions=3), home_id)
    X = pd.concat([f(gamestates) for f in [
        feat.actiontype_onehot, feat.result_onehot, feat.bodypart_onehot,
        feat.startlocation, feat.endlocation, feat.movement, feat.time
    ]], axis=1)
    # Ensure all expected model columns are present
    for col in expected_columns:
        if col not in X.columns:
            X[col] = 0
    X = X[expected_columns]
    actions["vaep_pred"] = model.predict(X)

    # Events enrichements , adding basic info to each action
    # time since previous action
    actions["time_diff"] = actions["time_seconds"].diff().fillna(0)
    # movement distance of the ball
    actions["distance"] = np.sqrt((actions['end_x'] - actions['start_x'])**2 + (actions['end_y'] - actions['start_y'])**2)

    # Select defensive actions (tackle, interception, clearance)
    defensive = actions[actions["type_name"].isin(["interception", "tackle", "clearance"])]

    # Identify crosses using event data
    events_flat = pd.json_normalize(events.to_dict(orient="records"), sep="_")
    passes_events = events_flat[events_flat["type_name"] == "Pass"]
    cross_events = passes_events[passes_events.get("extra_pass_cross", False) == True]

    # Define possessions
    actions = actions.sort_values(['team_id', 'period_id', 'time_seconds'])
    actions['possession_id'] = actions.groupby((actions['time_seconds'].diff() > 10).cumsum()).ngroup()
    actions['possession_time'] = actions.groupby(['team_id', 'possession_id'])['time_diff'].transform('sum')

    # Total number of actions per team (will be used in some metrics to improve them)
    total_actions = actions.groupby('team_id').size()

    # Ball regains
    regains = actions[actions['type_name'].isin(['interception', 'tackle', 'clearance'])]

    # Build-up speed , total distance moved per second of play
    if 'time_seconds' in actions.columns:
     actions['time_diff'] = actions.groupby(['team_id'])['time_seconds'].diff().fillna(0)
     df['BuildUpSpeed'] = actions.groupby('team_id')['distance'].sum() / actions.groupby('team_id')['time_diff'].sum()

    # Count aerial challenges (headed actions)
    df['AerialChallenges'] = actions[actions['bodypart_name'] == 'head'].groupby('team_id').size()

    # Initialize counters for counterattack metric
    counterattack_vaep = {home_id: 0, away_id: 0}
    counterattack_count = {home_id: 0, away_id: 0}
    opponent_counterattacks = {home_id: 0, away_id: 0}
    # Parameters for detecting counterattacks
    DEFENSIVE_ACTIONS = ['interception', 'tackle', 'clearance', 'keeper_save']
    MIN_PROGRESSION = 15  # meters of forward movement
    MAX_TIME_WINDOW = 10  # seconds
    MAX_ACTIONS_TO_CHECK = 5  # actions to examine
    
    # Detect counterattacks
    for idx, row in actions.iterrows():
        if row['type_name'] in DEFENSIVE_ACTIONS:
        
            team = row['team_id']
            opponent = home_id if team == away_id else away_id
            start_time = row['time_seconds']
            start_x = row['start_x']
            vaep_sum = 0
            max_progression = 0
            valid_counterattack = False
        
            # Check actions
            for i in range(1, MAX_ACTIONS_TO_CHECK + 1):
                if idx + i >= len(actions):
                    break
                
                next_row = actions.iloc[idx + i]
            
                # Break if possession changes or time window expires
                if next_row['team_id'] != team or (next_row['time_seconds'] - start_time) > MAX_TIME_WINDOW:
                    break
                
                # Track total VAEP value in the sequence
                vaep_sum += next_row['vaep_pred']
            
                # Track maximum progression achieved
                current_progression = next_row['end_x'] - start_x
                if current_progression > max_progression:
                    max_progression = current_progression
            
                # Early exit if we already have sufficient progression
                if max_progression >= MIN_PROGRESSION:
                    valid_counterattack = True

            
            # Register if we achieved minimum progression and positive VAEP
            if valid_counterattack and vaep_sum > 0:
                counterattack_vaep[team] += vaep_sum
                counterattack_count[team] += 1
                opponent_counterattacks[opponent] += 1

    # Save counterattack metrics, one for the team and one for the opponent
    df['CounterattackIndex'] = pd.Series(counterattack_vaep) / pd.Series(counterattack_count)
    OppCounterAttacks = pd.Series(opponent_counterattacks)
    defensiveCounterAttack = actions[actions["type_name"].isin(["interception", "tackle", "clearance"])].groupby('team_id').size()
    df['OppCounterAttackRate'] = OppCounterAttacks / defensiveCounterAttack

    # Defensive block compactness
    def_actions = actions[actions['type_name'].isin(['tackle', 'interception', 'clearance'])]
    def_total = def_actions.groupby('team_id').size()
    def_low = def_actions[def_actions['start_x'] <= 33.3].groupby('team_id').size()
    df['LowBlockRatio'] = def_low / def_total

    # Quick regains in middle third (preventing opponent counters)
    actions['team_change'] = actions['team_id'] != actions['team_id'].shift(1)
    actions['next_action_time'] = actions['time_seconds'].shift(-1)
    actions['time_to_next'] = actions['next_action_time'] - actions['time_seconds']
    quick_regains = actions[
        (actions['type_name'].isin(['interception', 'tackle'])) &
        (actions['start_x'] >= 33.3) & (actions['start_x'] <= 66.7) &
        (actions['time_to_next'] <= 5)  # quick regain after loss
    ]
    df['PreventedCounters_Mid3'] = quick_regains.groupby('team_id').size() / actions.groupby('team_id').size()

    # Count the number of crosses
    df["Crosses"] = cross_events.groupby("team_id").size()

    # Opponent attack types
    opponent_actions = actions.copy()
    df['OppOpenPlayRatio'] = opponent_actions[
        opponent_actions['type_name'].isin(['pass', 'dribble'])
    ].groupby('team_id').size() / opponent_actions.groupby('team_id').size()
    df['OppSetPieceRatio'] = opponent_actions[
        opponent_actions['type_name'].isin(['corner_crossed', 'freekick_crossed', 'freekick_short', 'throw_in'])
    ].groupby('team_id').size() / opponent_actions.groupby('team_id').size()

    # Where regains of the ball occur on the pitch (average X location)
    df['AvgDefensiveRegainX'] = regains.groupby('team_id')['start_x'].mean()

    # Pressing intensity (PPDA)
    passes = actions[actions['type_name'] == 'pass']
    defensive_actions = actions[
        (actions['type_name'].isin(['tackle', 'interception', 'clearance'])) & 
        (actions['start_x'] >= 40)
    ]
    ppda_n = passes.groupby('team_id').size()
    ppda_d = defensive_actions.groupby('team_id').size()
    df['PPDA'] = ppda_n / ppda_d

    # Defensive activity count
    df["DefensiveActions"] = defensive.groupby("team_id").size()

    # Play orientation indices
    df['CentralPlayIndex'] = actions[(actions['start_y'].between(30, 50))].groupby('team_id')['vaep_pred'].sum() / total_actions
    df['WingPlayIndex'] = actions[(actions['start_y'] < 20) | (actions['start_y'] > 60)].groupby('team_id')['vaep_pred'].sum() / total_actions

    # Dribble frequency
    df['Dribbles'] = actions[actions['type_name'] == 'dribble'].groupby('team_id').size() / actions.groupby('team_id').size()

    # Average time between a team’s actions
    df['TimePerAction'] = actions.groupby('team_id')['time_diff'].mean()

    # Lost balls via failed passes or dribbles
    lost_passes = actions[
        (actions['type_name'] == 'pass') & 
        (actions['result_name'] == 'fail')
    ]
    lost_dribbles = actions[
        (actions['type_name'].isin(['dribble', 'take_on'])) & 
        (actions['result_name'] == 'fail')
    ]
    # Combine and count
    lost_balls = pd.concat([lost_passes, lost_dribbles])
    df['LostBalls'] = lost_balls.groupby('team_id').size()

    # Defensive recoveries inside own half
    df['Recoveries_OwnHalf'] = actions[
        (actions['type_name'].isin(['interception', 'tackle', 'clearance'])) & 
        (actions['start_x'] < 50)
    ].groupby('team_id').size()

    # Fouls and yellow cards
    df['Fouls'] = actions[actions['type_name'] == 'foul'].groupby('team_id').size() / actions.groupby('team_id').size()
    if "extra_foul_committed_card_name" in events_flat.columns:
        carded_fouls = events_flat[
            events_flat["extra_foul_committed_card_name"].isin(["Yellow Card", "Red Card"])
        ]
        card_counts = carded_fouls.groupby(["team_id", "extra_foul_committed_card_name"]).size().unstack(fill_value=0)
        total_team_actions = actions.groupby("team_id").size()
        df["YellowCards"] = card_counts.get("Yellow Card", 0) / total_team_actions

    else:
        # if no yellow cards in the game
        df["YellowCards"] = 0


    # Creating Final Attempts (Little vs High Possession Needed)
    possesion_count = actions.groupby('team_id')['possession_id'].nunique()
    total_shots = actions[actions['type_name'] == 'shot'].groupby('team_id').size()
    df['FinalAttemptCreation'] = (total_shots / possesion_count.fillna(0))

    # Type of Attack (Set Pieces vs Open Play)
    set_piece_actions = actions[actions['type_name'].isin([
        'freekick_crossed', 'freekick_short', 'corner_crossed', 'corner_short'
    ])]
    set_piece_count = set_piece_actions.groupby('team_id').size()
    total_attacks = actions[actions['type_name'].isin(['pass', 'dribble', 'shot'])].groupby('team_id').size()
    df['SetPieceAttackRate'] = (set_piece_count / total_attacks).fillna(0)

    # Effective playing time (time when ball in play, ignoring >10s breaks)
    actions = actions.sort_values(['team_id', 'period_id', 'time_seconds'])
    actions['time_diff'] = actions.groupby(['team_id', 'period_id'])['time_seconds'].diff().fillna(0)
    actions['valid_play'] = actions['time_diff'] <= 10
    df['EffectiveTime'] = actions[actions['valid_play']].groupby('team_id')['time_diff'].sum()

    # fill missing values with 0
    df = df.fillna(0)
    # replace team ids with team names
    df.index = df.index.map(lambda x: home_name if x == home_id else away_name)
    return df

# store cumulative metrics for all teams
team_vaep_metrics = {}

# loop through each match
for _, row in tqdm(matches_of_interest.iterrows(), total=len(matches_of_interest)):
    game_id = row["game_id"]
    home_name = row["home_team_name"]
    away_name = row["away_team_name"]

    # get metrics for this match
    df = evaluate_team_metrics(game_id, home_name, away_name, model)

    # add metrics to each team (sum over matches)
    for team in df.index:
        if team not in team_vaep_metrics:
            team_vaep_metrics[team] = df.loc[team]  # first time just add
        else:
            team_vaep_metrics[team] += df.loc[team] # otherwise sum up

# convert dictionary to DataFrame for readability
df_stocked = pd.DataFrame(team_vaep_metrics).T.fillna(0)
print("Data available in df_stocked")
print(df_stocked)


def apply_pca_transformation(df_stocked, n_components=None, variance_threshold=0.95):
    """
    Apply PCA to create latent tactical style components
    
    """
    print(" PCA-BASED TACTICAL STYLE TRANSFORMATION")
    
    # prepare data
    X = df_stocked.copy()
    # make sure to have only one team and no duplicates
    X = X.loc[~X.index.duplicated(keep='first')]

    # CORRELATION ANALYSIS
    print("\n CORRELATION ANALYSIS")
    print("-" * 50)
    
    corr_matrix = X.corr()
    
    # find highly correlated features
    high_corr_pairs = []
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            if abs(corr_matrix.iloc[i, j]) > 0.7:
                high_corr_pairs.append((
                    corr_matrix.columns[i], 
                    corr_matrix.columns[j], 
                    corr_matrix.iloc[i, j]
                ))
    
    print(f"Highly correlated pairs (>0.7): {len(high_corr_pairs)}")
    for feat1, feat2, corr in high_corr_pairs:
        print(f"   {feat1} ↔ {feat2}: {corr:.3f}")
    
    # STANDARDIZATION
    print("\n DATA STANDARDIZATION")
    print("-" * 50)
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    print(" Data standardized (mean=0, std=1)")
    
    # PCA APPLICATION
    print("\n PCA APPLICATION")
    print("-" * 50)
    
    # apply PCA with all components first to analyze variance
    pca_full = PCA()
    X_pca_full = pca_full.fit_transform(X_scaled)
    
    # calculate cumulative variance
    added_variance = np.cumsum(pca_full.explained_variance_ratio_)
    
    # determine the number of components
    if n_components is None:
        n_components = np.argmax(added_variance >= variance_threshold) + 1
        print(f"Components needed for {variance_threshold*100}% variance: {n_components}")
    
    # apply PCA with the chosen number of components
    pca = PCA(n_components=n_components, random_state=42)
    X_pca = pca.fit_transform(X_scaled)
    
    print(f"Final PCA shape: {X_pca.shape}")
    print(f"Explained variance ratio: {pca.explained_variance_ratio_}")
    print(f"Cumulative variance explained: {np.sum(pca.explained_variance_ratio_):.3f}")
    
    # component interpretation
    print("\n TACTICAL STYLE COMPONENTS INTERPRETATION")
    print("-" * 50)
    
    # create component interpretation
    components_df = pd.DataFrame(
        pca.components_.T,
        columns=[f'Style_{i+1}' for i in range(n_components)],
        index=X.columns
    )
    
    # interpret each component
    tactical_styles = {}
    for i in range(n_components):
        component_name = f'Style_{i+1}'
        component_loadings = components_df[component_name].abs().sort_values(ascending=False)
        
        print(f"\n {component_name} (explains {pca.explained_variance_ratio_[i]:.1%} variance):")
        
        # top positive loadings
        pos_loadings = components_df[component_name].sort_values(ascending=False).head(3)
        print("   Top positive loadings:")
        for feat, loading in pos_loadings.items():
            print(f"      +{loading:.3f} {feat}")
        
        # top negative loadings
        neg_loadings = components_df[component_name].sort_values(ascending=True).head(3)
        print("   Top negative loadings:")
        for feat, loading in neg_loadings.items():
            print(f"      {loading:.3f} {feat}")
        
        # suggest a tactical interpretation
        top_features = component_loadings.head(3).index.tolist()
        tactical_styles[component_name] = {
            'variance_explained': pca.explained_variance_ratio_[i],
            'top_features': top_features
        }
    
    # Create a PCA dataFrame for visibility and better understandings
    pca_df = pd.DataFrame(
        X_pca, 
        columns=[f'Style_{i+1}' for i in range(n_components)],
        index=X.index
    )
    
    return {
        'pca_df': pca_df,
        'pca_model': pca,
        'scaler': scaler,
        'components_df': components_df,
        'tactical_styles': tactical_styles,
        'original_data': X,
        'scaled_data': X_scaled,
        'explained_variance_ratio': pca.explained_variance_ratio_,
        'added_variance': np.cumsum(pca.explained_variance_ratio_)
    }

def cluster_pca_styles(pca_results, clustering_methods=None, max_clusters=10):
    """
    Apply clustering to PCA-transformed tactical styles
    """
    # The 5 clustering algorithms
    if clustering_methods is None:
        clustering_methods = {
            "KMeans": {"n_clusters": 9},
            "Agglomerative": {"n_clusters": 9},
            "GMM": {"n_components": 9},
            "Spectral": {"n_clusters": 9},
            "Birch": {"n_clusters": 6}
        }
    

    # Extract the PCA-transformed feature matrix (scores) from results
    # Get PCA scores as a NumPy array for clustering
    pca_df = pca_results['pca_df']
    X_pca = pca_df.values
    
    # standardize PCA components
    scaler_pca = StandardScaler()
    X_pca_scaled = scaler_pca.fit_transform(X_pca)


    print("\n OPTIMAL NUMBER OF CLUSTERS")
    print("-" * 50)

    # Test different numbers of clusters
    cluster_range = range(2, max_clusters + 1)

    # metrics for different clustering algorithms
    metrics = {
        'KMeans': {'silhouette': [], 'calinski': [], 'davies': []},
        'Agglomerative': {'silhouette': [], 'calinski': [], 'davies': []},
        'GMM': {'silhouette': [], 'calinski': [], 'davies': []},
        'Birch': {'silhouette': [], 'calinski': [], 'davies': []},
        'Spectral': {'silhouette': [], 'calinski': [], 'davies': []}
    }

    # Try different numbers of clusters for each algorithm and record quality metrics (Silhouette, Calinski-Harabasz, Davies-Bouldin)
    for n_clusters in cluster_range:
        
        # KMeans
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        labels_km = kmeans.fit_predict(X_pca_scaled)
        
        # Agglomerative
        agg = AgglomerativeClustering(n_clusters=n_clusters)
        labels_agg = agg.fit_predict(X_pca_scaled)
        
        # GMM
        gmm = GaussianMixture(n_components=n_clusters, random_state=42)
        labels_gmm = gmm.fit_predict(X_pca_scaled)
        
        # Birch 
        birch = Birch(n_clusters=n_clusters)
        labels_birch = birch.fit_predict(X_pca_scaled)
        
        # Spectral
        try:
            spectral = SpectralClustering(n_clusters=n_clusters, random_state=42, affinity='nearest_neighbors')
            labels_spectral = spectral.fit_predict(X_pca_scaled)
        except Exception as e:
            print(f"  Spectral clustering failed for {n_clusters} clusters: {e}")
            # créer des labels par défaut en cas d'échec
            labels_spectral = np.zeros(len(X_pca_scaled), dtype=int)
        
        # calculate metrics 
        for name, labels in [('KMeans', labels_km), ('Agglomerative', labels_agg), 
                           ('GMM', labels_gmm), ('Birch', labels_birch), ('Spectral', labels_spectral)]:
            if len(np.unique(labels)) > 1:  # Need at least 2 clusters for metrics (For 2D vizualisation)
                try:
                    metrics[name]['silhouette'].append(silhouette_score(X_pca_scaled, labels))
                    metrics[name]['calinski'].append(calinski_harabasz_score(X_pca_scaled, labels))
                    metrics[name]['davies'].append(davies_bouldin_score(X_pca_scaled, labels))
                except Exception as e:
                    print(f"   Error calculating metrics for {name} with {n_clusters} clusters: {e}")
                    metrics[name]['silhouette'].append(0)
                    metrics[name]['calinski'].append(0)
                    metrics[name]['davies'].append(float('inf'))
            else:
                metrics[name]['silhouette'].append(0)
                metrics[name]['calinski'].append(0)
                metrics[name]['davies'].append(float('inf'))

    # print optimal number of clusters
    print("Optimal number of clusters by metric:")
    for algorithm in metrics.keys():
        if metrics[algorithm]['silhouette']:
            try:
                best_sil = cluster_range[np.argmax(metrics[algorithm]['silhouette'])]
                best_cal = cluster_range[np.argmax(metrics[algorithm]['calinski'])]
                best_dav = cluster_range[np.argmin(metrics[algorithm]['davies'])]
                print(f"   {algorithm:>12}: Silhouette={best_sil}, Calinski={best_cal}, Davies-Bouldin={best_dav}")
            except Exception as e:
                print(f"   {algorithm:>12}: Error calculating optimal clusters - {e}")

    # Print the full score table per algorithm
    print("\n Full clustering scores by number of clusters:")
    for algorithm in metrics:
        print(f"\n {algorithm}")
        print("Clusters | Silhouette | Calinski-Harabasz | Davies-Bouldin")
        for i, n in enumerate(cluster_range):
            if i < len(metrics[algorithm]['silhouette']):
                sil = metrics[algorithm]['silhouette'][i]
                cal = metrics[algorithm]['calinski'][i]
                dav = metrics[algorithm]['davies'][i]
                print(f"{n:>8} | {sil:>10.3f} | {cal:>18.2f} | {dav:>15.3f}")
    
    clustering_results = {}

    # Fit each clustering algorithm with its chosen parameters on the PCA data
    # compute quality metrics (Silhouette, Calinski-Harabasz, Davies-Bouldin)
    # store the results and print the composition of each cluster.
    for method_name, params in clustering_methods.items():
        print(f"\n {method_name} Clustering")
        
        try:
            if method_name == "KMeans":
                model = KMeans(random_state=42, n_init=10, **params)
                labels = model.fit_predict(X_pca_scaled)
                
            elif method_name == "Agglomerative":
                model = AgglomerativeClustering(**params)
                labels = model.fit_predict(X_pca_scaled)
                
            elif method_name == "GMM":
                model = GaussianMixture(random_state=42, **params)
                labels = model.fit_predict(X_pca_scaled)
                
            elif method_name == "Spectral":
                model = SpectralClustering(random_state=42, affinity='nearest_neighbors', **params)
                labels = model.fit_predict(X_pca_scaled)
                
            elif method_name == "Birch":  
                model = Birch(**params)
                labels = model.fit_predict(X_pca_scaled)
            
            # evaluate clustering quality
            if len(np.unique(labels)) > 1:
                sil_score = silhouette_score(X_pca_scaled, labels)
                cal_score = calinski_harabasz_score(X_pca_scaled, labels)
                dav_score = davies_bouldin_score(X_pca_scaled, labels)
                
                print(f"   Silhouette Score: {sil_score:.3f}")
                print(f"   Calinski-Harabasz: {cal_score:.2f}")
                print(f"   Davies-Bouldin: {dav_score:.3f}")
                
                clustering_results[method_name] = {
                    'labels': labels,
                    'model': model,
                    'silhouette': sil_score,
                    'calinski': cal_score,
                    'davies_bouldin': dav_score
                }
                
                # Print cluster composition
                print(f"   Cluster composition:")
                for cluster_id in np.unique(labels):
                    cluster_teams = pca_df.index[labels == cluster_id].tolist()
                    print(f"      Cluster {cluster_id}: {cluster_teams}")
                    
        except Exception as e:
            print(f" Error with {method_name}: {e}")
    
    return clustering_results


def visualize_pca_clusters(pca_results, clustering_results):
    """
    Visualize clustering results on PCA space
    """
    print("\n CLUSTERING VISUALIZATION")
    print("-" * 50)

    #Extract PCA scores (teams and components) and variance ratios for labeling axes
    pca_df = pca_results['pca_df']
    explained_variance = pca_results['explained_variance_ratio']
    
    # create a visualization for each clustering method
    for method_name, result in clustering_results.items():
        labels = result['labels']
        sil_score = result['silhouette']
        
        # create plot dataframe
        plot_df = pca_df.copy()
        plot_df['Team'] = plot_df.index
        plot_df['Cluster'] = labels.astype(str)
        
        # 2D visualization for first two components
        if pca_df.shape[1] >= 2:
            fig = px.scatter(
                plot_df, x='Style_1', y='Style_2',
                color='Cluster', text='Team',
                category_orders={"Cluster": sorted(plot_df["Cluster"].unique(), key=lambda x: int(x))}, 
                title=f'{method_name} Clustering on Tactical Styles',
                labels={'Style_1': f'Style 1 ({explained_variance[0]:.1%})',
                       'Style_2': f'Style 2 ({explained_variance[1]:.1%})'},
                width=1000, height=600
            )
            fig.update_traces(textposition='top center')
            fig.show()
        
        # 3D visualization if we have at least 3 components
        if pca_df.shape[1] >= 3:
            fig = px.scatter_3d(
                plot_df, x='Style_1', y='Style_2', z='Style_3',
                color='Cluster', text='Team',
                category_orders={"Cluster": sorted(plot_df["Cluster"].unique(), key=lambda x: int(x))}, 
                title=f'{method_name} Clustering - 3D Tactical Space',
                labels={'Style_1': f'Style 1 ({explained_variance[0]:.1%})',
                       'Style_2': f'Style 2 ({explained_variance[1]:.1%})',
                       'Style_3': f'Style 3 ({explained_variance[2]:.1%})'},
                width=1000, height=700
            )
            fig.show()

def compute_global_discriminant_features(pca_results, clustering_results):
    """
    Compute average F-ratio (feature importance) across all clustering methods.
    """
    print("\n GLOBAL MOST DISCRIMINANT FEATURES (FOR ALL CLUSTERINGS)")
    print("=" * 60)

    # Prepare a dictionary to store F-ratios per feature for each clustering method
    original_data = pca_results['original_data']
    global_f_ratios = {feature: [] for feature in original_data.columns}

    # Loop over results from each clustering algorithm
    for method_name, result in clustering_results.items():
        labels = result['labels']
        original_with_clusters = original_data.copy()
        original_with_clusters['Cluster'] = labels
        unique_clusters = np.unique(labels)

        #Loop through every original metric (feature)
        for feature in original_data.columns:
            feature_values = original_with_clusters[feature].values
            overall_mean = np.mean(feature_values)

            between_ss = 0
            within_ss = 0

            # Loop through each cluster to accumulate between/within variance
            for cluster_id in unique_clusters:
                cluster_vals = original_with_clusters[original_with_clusters['Cluster'] == cluster_id][feature].values
                cluster_mean = np.mean(cluster_vals)
                between_ss += len(cluster_vals) * (cluster_mean - overall_mean) ** 2
                within_ss += np.sum((cluster_vals - cluster_mean) ** 2)

            # Compute the F-ratio: higher means the feature better separates clusters
            f_ratio = between_ss / within_ss if within_ss > 0 else 0
            # Save this F-ratio for the current feature and clustering method
            global_f_ratios[feature].append(f_ratio)

    # average F-ratios across algorithms
    avg_f_ratios = [(feature, np.mean(f_vals)) for feature, f_vals in global_f_ratios.items()]
    avg_f_ratios.sort(key=lambda x: x[1], reverse=True)

    print(" Averaged most discriminating features across clustering methods:")
    for i, (feature, avg_f) in enumerate(avg_f_ratios):
        print(f"   {i+1:2d}. {feature:<25} Avg F-ratio = {avg_f:.3f}")

    # main execution function
def run_pca_tactical_analysis(df_stocked):
    """
    Run a complete PCA-based tactical analysis
    """
    # 1: Apply PCA transformation
    pca_results = apply_pca_transformation(df_stocked, variance_threshold=0.85)
    
    # 2: Apply clustering to PCA components
    clustering_results = cluster_pca_styles(pca_results, max_clusters=9)
    
    # 3: Visualize clustering results
    visualize_pca_clusters(pca_results, clustering_results)

    # 4: Compute global discriminant features across all clusterings
    compute_global_discriminant_features(pca_results, clustering_results)

    #Return results for further analysis
    return {
        'pca_results': pca_results,
        'clustering_results': clustering_results
    }


# Run main function
results = run_pca_tactical_analysis(df_stocked)

#--------------------------------------------------------------------------------------------------------
#--------------------------------------------------------------------------------------------------------
#--------------------------------------------------------------------------------------------------------
# Additional functionalities if needed

# save the scores per team based on average with colors in images

# calculate average and categorize performance with 6 levels
metric_averages = df_stocked.mean()
performance = pd.DataFrame(index=df_stocked.index, columns=df_stocked.columns)

# Assign performance ratings for each team and metric using league average ± standard deviations
for col in df_stocked.columns:
    avg = metric_averages[col]
    std = df_stocked[col].std()
    for team in df_stocked.index:
        val = df_stocked.loc[team, col]
        if val >= avg + 2*std:
            cat = "+++"  # very good performance (2+ std above avg)
        elif val >= avg + std:
            cat = "++"   # good performance (1-2 std above avg)
        elif val >= avg:
            cat = "+"    # Above average performance
        elif val <= avg - 2*std:
            cat = "---"  # Very poor performance (2+ std below avg)
        elif val <= avg - std:
            cat = "--"   # Poor performance (1-2 std below avg)
        else:
            cat = "-"    # Below average performance
        performance.loc[team, col] = cat

# combine value and label
df_display = df_stocked.round(2).astype(str) + " (" + performance + ")"
df_display.loc["League Average"] = [f"{metric_averages[col]:.2f} (avg)" for col in df_stocked.columns]

# save Horizontal Table ----------
plt.figure(figsize=(len(df_display.columns) * 2.2, len(df_display) * 0.35))
sns.set(font_scale=0.72)
table = plt.table(cellText=df_display.values,
                  rowLabels=df_display.index,
                  colLabels=df_display.columns,
                  loc='center',
                  cellLoc='center')
table.auto_set_font_size(False)
table.set_fontsize(6.5)
table.scale(1.05, 1.08)

# add colors based on performance categories (6 levels)
for i in range(len(df_display)):
    for j in range(len(df_display.columns)):
        cell_text = df_display.iloc[i, j]
        if "(+++)" in cell_text:
            table[(i+1, j)].set_facecolor('#006400')  # Very dark green
            table[(i+1, j)].set_text_props(weight='bold', color='white')
        elif "(++)" in cell_text:
            table[(i+1, j)].set_facecolor('#2E8B57')  # Dark green
            table[(i+1, j)].set_text_props(weight='bold', color='white')
        elif "(+)" in cell_text:
            table[(i+1, j)].set_facecolor('#90EE90')  # Light green
            table[(i+1, j)].set_text_props(weight='bold')
        elif "(---)" in cell_text:
            table[(i+1, j)].set_facecolor('#8B0000')  # Very dark red
            table[(i+1, j)].set_text_props(weight='bold', color='white')
        elif "(--)" in cell_text:
            table[(i+1, j)].set_facecolor('#CD5C5C')  # Dark red
            table[(i+1, j)].set_text_props(weight='bold', color='white')
        elif "(-)" in cell_text:
            table[(i+1, j)].set_facecolor('#FFB6C1')  # Light red
            table[(i+1, j)].set_text_props(weight='bold')
        elif "(avg)" in cell_text:
            table[(i+1, j)].set_facecolor('#D3D3D3')  # Light gray
            table[(i+1, j)].set_text_props(weight='bold')

plt.title('Team Tactical Metrics - Horizontal View', fontsize=14, fontweight='bold', pad=20)
plt.axis('off')
#plt.savefig("team_tactical_metrics_performance_table_Ligue1_1516.png", bbox_inches='tight', dpi=300)


print("\nLegend:")
print("+++ (Very Dark Green): Exceptional - Above average + 2 standard deviations")
print("++ (Dark Green): Very Good - Above average + 1 standard deviation")
print("+ (Light Green): Good - Above average")
print("- (Light Red): Below average")
print("-- (Dark Red): Poor - Below average - 1 standard deviation")
print("--- (Very Dark Red): Very Poor - Below average - 2 standard deviations")
print("avg (Gray): League average")

# Display the categorized data 
print("\n Performance Categories:")
print(performance)

# Display average scores per metric
print("\n Average Scores per Metric:")
print(metric_averages.round(2))


#---------------------------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------------------------

# If you want to create 5 distinctive horizontal tables with the performance scores instead of one big image like above
# Define metric groups
#metric_groups = {
#    "Metrics 1-5": df_display.iloc[:, 0:5],
#    "Metrics 6-10": df_display.iloc[:, 5:10], 
#    "Metrics 11-15": df_display.iloc[:, 10:15],
#    "Metrics 16-20": df_display.iloc[:, 15:20],
#    "Metrics 21-23": df_display.iloc[:, 20:23]
#}

#def create_colored_table(df_subset, title, filename):
#    """Create a colored table for a subset of metrics"""
#    plt.figure(figsize=(len(df_subset.columns) * 2.2, len(df_subset) * 0.35))
#    sns.set(font_scale=0.72)
    
#    table = plt.table(cellText=df_subset.values,
#                      rowLabels=df_subset.index,  # Team names on the side
#                      colLabels=df_subset.columns,
#                      loc='center',
#                      cellLoc='center')
    
#    table.auto_set_font_size(False)
#    table.set_fontsize(6.5)
#    table.scale(1.05, 1.08)

    # Add colors based on performance categories (5 levels)
#    for i in range(len(df_subset)):
#        for j in range(len(df_subset.columns)):
#            cell_text = df_subset.iloc[i, j]
#            if "(+++)" in str(cell_text):
#                table[(i+1, j)].set_facecolor('#006400')  # Very dark green
#                table[(i+1, j)].set_text_props(weight='bold', color='white')
#            elif "(++)" in str(cell_text):
#                table[(i+1, j)].set_facecolor('#2E8B57')  # Dark green
#                table[(i+1, j)].set_text_props(weight='bold', color='white')
#            elif "(+)" in str(cell_text):
#                table[(i+1, j)].set_facecolor('#90EE90')  # Light green
#                table[(i+1, j)].set_text_props(weight='bold')
#            elif "(---)" in str(cell_text):
#                table[(i+1, j)].set_facecolor('#8B0000')  # Very dark red
#                table[(i+1, j)].set_text_props(weight='bold', color='white')
#            elif "(--)" in str(cell_text):
#                table[(i+1, j)].set_facecolor('#CD5C5C')  # Dark red
#                table[(i+1, j)].set_text_props(weight='bold', color='white')
#            elif "(-)" in str(cell_text):
#                table[(i+1, j)].set_facecolor('#FFB6C1')  # Light red
#                table[(i+1, j)].set_text_props(weight='bold')
#            elif "(avg)" in str(cell_text):
#                table[(i+1, j)].set_facecolor('#D3D3D3')  # Light gray
#                table[(i+1, j)].set_text_props(weight='bold')

#    plt.title(f'Team Tactical Metrics - {title}', fontsize=14, fontweight='bold', pad=20)
#    plt.axis('off')
#    plt.savefig(filename, bbox_inches='tight', dpi=300)
#    plt.show()
#    print(f"Created: {filename}")

# Create all 5 tables
#for group_name, df_subset in metric_groups.items():
#    filename = f"tactical_metrics_{group_name.lower().replace(' ', '_').replace('-', '_')}.png"
#    create_colored_table(df_subset, group_name, filename)

## EFA-related indicators 

In [ ]:
import joblib
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
import plotly.express as px
from socceraction.data.statsbomb import StatsBombLoader
from socceraction.spadl import statsbomb as spadl_sb
from socceraction import spadl
from socceraction.vaep import features as feat
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering, DBSCAN, Birch
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

#Load StatsBomb data locally
sbl = StatsBombLoader(
    getter="local",  # from local files
    root="C:/Users/helio/MachineLearning/Bachelor/statsbomb_data/open-data-master/data"
)

# Get all games for a given competition and season
# Competition ID = 2, Season ID = 27 (Premier League 2015/16)
# Competition ID = 11, Season ID = 27 (La Liga 2015/16)
# Competition ID = 9, Season ID = 27 (Bundesliga 2015/16)
# Competition ID = 12, Season ID = 27 (Serie A 2015/16)
# Competition ID = 7, Season ID = 27 (Ligue 1 2015/16)
games = sbl.games(2, 27)

# Load pre-trained VAEP model that some metrics need, especially VAEP metrics
data = joblib.load("C:/Users/helio/MachineLearning/Bachelor/vaep_model_xgboost_5leagues_1516.pkl")
# The trained model
model = data["model"]          
# Expected input features for model
expected_columns = data["feature_columns"] 


# Create a mapping of team ids to names for all matches of interest
matches_of_interest = games[["game_id", "home_team_id", "away_team_id"]].copy()
team_id_name_map = {}
for game_id in matches_of_interest["game_id"]:
    teams = sbl.teams(game_id)
    for _, row in teams.iterrows():
        team_id_name_map[row["team_id"]] = row["team_name"]

# Add team names to match dataframe
matches_of_interest["home_team_name"] = matches_of_interest["home_team_id"].map(team_id_name_map)
matches_of_interest["away_team_name"] = matches_of_interest["away_team_id"].map(team_id_name_map)

# Keep only rows with valid team names
matches_of_interest = matches_of_interest.dropna(subset=["home_team_name", "away_team_name"])


# compute VAEP-based team metrics for a single match
def evaluate_team_metrics(game_id, home_name, away_name, model):
    # Load game and prepare base dataframe
    match = games.loc[games["game_id"] == game_id].iloc[0]
    home_id, away_id = match["home_team_id"], match["away_team_id"]
    events = sbl.events(game_id)
    # convert to SPADL actions
    actions = spadl.add_names(spadl_sb.convert_to_actions(events, home_id))
    df = pd.DataFrame(index=[home_id, away_id])

    # Predict VAEP
    gamestates = feat.play_left_to_right(feat.gamestates(actions, nb_prev_actions=3), home_id)
    X = pd.concat([f(gamestates) for f in [
        feat.actiontype_onehot, feat.result_onehot, feat.bodypart_onehot,
        feat.startlocation, feat.endlocation, feat.movement, feat.time
    ]], axis=1)
    for col in expected_columns:
        if col not in X.columns:
            X[col] = 0
    X = X[expected_columns]
    actions["vaep_pred"] = model.predict(X)


    # 1. Possession
    actions["time_diff"] = actions["time_seconds"].diff().fillna(0)
    total_time = actions.groupby("team_id")["time_diff"].sum()
    df["Possession"] = total_time / total_time.sum()

    # Field thirds & zones
    defensive_third = actions["start_x"] <= 33.3
    middle_third = (actions["start_x"] > 33.3) & (actions["start_x"] <= 66.6)
    attacking_third = actions["start_x"] > 66.6
    central_area = actions["start_y"].between(30, 50)
    wide_area = (actions["start_y"] < 20) | (actions["start_y"] > 60)
    team_time_possession = actions.groupby("team_id")["time_diff"].sum()

    # 2–6: possession in different areas
    df["PossDef3"] = actions[defensive_third].groupby("team_id")["time_diff"].sum() / team_time_possession
    df["PossMid3"] = actions[middle_third].groupby("team_id")["time_diff"].sum() / team_time_possession
    df["PossAtt3"] = actions[attacking_third].groupby("team_id")["time_diff"].sum() / team_time_possession
    df["PossCentral"] = actions[central_area].groupby("team_id")["time_diff"].sum() / team_time_possession
    df["PossWide"] = actions[wide_area].groupby("team_id")["time_diff"].sum() / team_time_possession

    # 7–10: Pass directions
    passes = actions[actions["type_name"] == "pass"]
    dx = passes["end_x"] - passes["start_x"]
    dy = passes["end_y"] - passes["start_y"]

    def pass_direction(dx, dy):
        if abs(dx) < abs(dy) and abs(dy) > 5:  # sideways
            return "side"
        elif dx > 5:
            return "forward"
        elif dx < -5:
            return "backward"
        else:
            return "side"

    passes["direction"] = [pass_direction(x, y) for x, y in zip(dx, dy)]
    direction_score = passes["direction"].map({"backward": 1, "side": 2, "forward": 3})

    df["PassDirectionScore"] = passes.groupby("team_id")["direction"].apply(lambda s: s.map({"backward":1,"side":2,"forward":3}).mean())
    df["ForwardPassPct"] = (passes["direction"] == "forward").groupby(passes["team_id"]).mean()
    df["SidePassPct"] = (passes["direction"] == "side").groupby(passes["team_id"]).mean()
    df["BackPassPct"] = (passes["direction"] == "backward").groupby(passes["team_id"]).mean()

    # 11–12: passes between thirds
    from_def_to_mid = defensive_third & (passes["end_x"] > 33.3) & (passes["end_x"] <= 66.6)
    from_def_to_att = defensive_third & (passes["end_x"] > 66.6)
    df["PassDefToMid"] = from_def_to_mid.groupby(passes["team_id"]).mean()
    df["PassDefToAtt"] = from_def_to_att.groupby(passes["team_id"]).mean()

    # 13: Crosses 
    crosses = passes[passes["type_name"] == "cross"]
    df["CrossPct"] = crosses.groupby("team_id").size() / passes.groupby("team_id").size()

    # 14: Shots 
    shots = actions[actions["type_name"] == "shot"]
    df["ShotPct"] = shots.groupby("team_id").size() / passes.groupby("team_id").size()

    # 15–19: Regains 
    regains = actions[actions["type_name"].isin(["interception", "tackle", "clearance"])]
    total_regains = regains.groupby("team_id").size()

    df["RegainDef3"] = regains[defensive_third].groupby("team_id").size() / total_regains
    df["RegainMid3"] = regains[middle_third].groupby("team_id").size() / total_regains
    df["RegainAtt3"] = regains[attacking_third].groupby("team_id").size() / total_regains
    df["RegainCentral"] = regains[central_area].groupby("team_id").size() / total_regains
    df["RegainWide"] = regains[wide_area].groupby("team_id").size() / total_regains

    # fill missing values with 0
    df = df.fillna(0)
    # replace team ids with team names
    df.index = df.index.map(lambda x: home_name if x == home_id else away_name)
    return df

# store cumulative metrics for all teams
team_vaep_metrics = {}

# loop through each match
for _, row in tqdm(matches_of_interest.iterrows(), total=len(matches_of_interest)):
    game_id = row["game_id"]
    home_name = row["home_team_name"]
    away_name = row["away_team_name"]

    # get metrics for this match
    df = evaluate_team_metrics(game_id, home_name, away_name, model)

    # add metrics to each team (sum over matches)
    for team in df.index:
        if team not in team_vaep_metrics:
            team_vaep_metrics[team] = df.loc[team]  # first time just add
        else:
            team_vaep_metrics[team] += df.loc[team] # otherwise sum up

# convert dictionary to DataFrame for readability
df_stocked = pd.DataFrame(team_vaep_metrics).T.fillna(0)
print("Data available in df_stocked")
print(df_stocked)


def apply_pca_transformation(df_stocked, n_components=None, variance_threshold=0.95):
    """
    Apply PCA to create latent tactical style components
    
    """
    print(" PCA-BASED TACTICAL STYLE TRANSFORMATION")
    
    # prepare data
    X = df_stocked.copy()
    # make sure to have only one team and no duplicates
    X = X.loc[~X.index.duplicated(keep='first')]

    # CORRELATION ANALYSIS
    print("\n CORRELATION ANALYSIS")
    print("-" * 50)
    
    corr_matrix = X.corr()
    
    # find highly correlated features
    high_corr_pairs = []
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            if abs(corr_matrix.iloc[i, j]) > 0.7:
                high_corr_pairs.append((
                    corr_matrix.columns[i], 
                    corr_matrix.columns[j], 
                    corr_matrix.iloc[i, j]
                ))
    
    print(f"Highly correlated pairs (>0.7): {len(high_corr_pairs)}")
    for feat1, feat2, corr in high_corr_pairs:
        print(f"   {feat1} ↔ {feat2}: {corr:.3f}")
    
    # STANDARDIZATION
    print("\n DATA STANDARDIZATION")
    print("-" * 50)
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    print(" Data standardized (mean=0, std=1)")
    
    # PCA APPLICATION
    print("\n PCA APPLICATION")
    print("-" * 50)
    
    # apply PCA with all components first to analyze variance
    pca_full = PCA()
    X_pca_full = pca_full.fit_transform(X_scaled)
    
    # calculate cumulative variance
    added_variance = np.cumsum(pca_full.explained_variance_ratio_)
    
    # determine the number of components
    if n_components is None:
        n_components = np.argmax(added_variance >= variance_threshold) + 1
        print(f"Components needed for {variance_threshold*100}% variance: {n_components}")
    
    # apply PCA with the chosen number of components
    pca = PCA(n_components=n_components, random_state=42)
    X_pca = pca.fit_transform(X_scaled)
    
    print(f"Final PCA shape: {X_pca.shape}")
    print(f"Explained variance ratio: {pca.explained_variance_ratio_}")
    print(f"Cumulative variance explained: {np.sum(pca.explained_variance_ratio_):.3f}")
    
    # component interpretation
    print("\n TACTICAL STYLE COMPONENTS INTERPRETATION")
    print("-" * 50)
    
    # create component interpretation
    components_df = pd.DataFrame(
        pca.components_.T,
        columns=[f'Style_{i+1}' for i in range(n_components)],
        index=X.columns
    )
    
    # interpret each component
    tactical_styles = {}
    for i in range(n_components):
        component_name = f'Style_{i+1}'
        component_loadings = components_df[component_name].abs().sort_values(ascending=False)
        
        print(f"\n {component_name} (explains {pca.explained_variance_ratio_[i]:.1%} variance):")
        
        # top positive loadings
        pos_loadings = components_df[component_name].sort_values(ascending=False).head(3)
        print("   Top positive loadings:")
        for feat, loading in pos_loadings.items():
            print(f"      +{loading:.3f} {feat}")
        
        # top negative loadings
        neg_loadings = components_df[component_name].sort_values(ascending=True).head(3)
        print("   Top negative loadings:")
        for feat, loading in neg_loadings.items():
            print(f"      {loading:.3f} {feat}")
        
        # suggest a tactical interpretation
        top_features = component_loadings.head(3).index.tolist()
        tactical_styles[component_name] = {
            'variance_explained': pca.explained_variance_ratio_[i],
            'top_features': top_features
        }
    
    # Create a PCA dataFrame for visibility and better understandings
    pca_df = pd.DataFrame(
        X_pca, 
        columns=[f'Style_{i+1}' for i in range(n_components)],
        index=X.index
    )
    
    return {
        'pca_df': pca_df,
        'pca_model': pca,
        'scaler': scaler,
        'components_df': components_df,
        'tactical_styles': tactical_styles,
        'original_data': X,
        'scaled_data': X_scaled,
        'explained_variance_ratio': pca.explained_variance_ratio_,
        'added_variance': np.cumsum(pca.explained_variance_ratio_)
    }

def cluster_pca_styles(pca_results, clustering_methods=None, max_clusters=10):
    """
    Apply clustering to PCA-transformed tactical styles
    """
    # The 5 clustering algorithms
    if clustering_methods is None:
        clustering_methods = {
            "KMeans": {"n_clusters": 9},
            "Agglomerative": {"n_clusters": 9},
            "GMM": {"n_components": 9},
            "Spectral": {"n_clusters": 9},
            "Birch": {"n_clusters": 6}
        }
    

    # Extract the PCA-transformed feature matrix (scores) from results
    # Get PCA scores as a NumPy array for clustering
    pca_df = pca_results['pca_df']
    X_pca = pca_df.values
    
    # standardize PCA components
    scaler_pca = StandardScaler()
    X_pca_scaled = scaler_pca.fit_transform(X_pca)


    print("\n OPTIMAL NUMBER OF CLUSTERS")
    print("-" * 50)

    # Test different numbers of clusters
    cluster_range = range(2, max_clusters + 1)

    # metrics for different clustering algorithms
    metrics = {
        'KMeans': {'silhouette': [], 'calinski': [], 'davies': []},
        'Agglomerative': {'silhouette': [], 'calinski': [], 'davies': []},
        'GMM': {'silhouette': [], 'calinski': [], 'davies': []},
        'Birch': {'silhouette': [], 'calinski': [], 'davies': []},
        'Spectral': {'silhouette': [], 'calinski': [], 'davies': []}
    }

    # Try different numbers of clusters for each algorithm and record quality metrics (Silhouette, Calinski-Harabasz, Davies-Bouldin)
    for n_clusters in cluster_range:
        
        # KMeans
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        labels_km = kmeans.fit_predict(X_pca_scaled)
        
        # Agglomerative
        agg = AgglomerativeClustering(n_clusters=n_clusters)
        labels_agg = agg.fit_predict(X_pca_scaled)
        
        # GMM
        gmm = GaussianMixture(n_components=n_clusters, random_state=42)
        labels_gmm = gmm.fit_predict(X_pca_scaled)
        
        # Birch 
        birch = Birch(n_clusters=n_clusters)
        labels_birch = birch.fit_predict(X_pca_scaled)
        
        # Spectral
        try:
            spectral = SpectralClustering(n_clusters=n_clusters, random_state=42, affinity='nearest_neighbors')
            labels_spectral = spectral.fit_predict(X_pca_scaled)
        except Exception as e:
            print(f"  Spectral clustering failed for {n_clusters} clusters: {e}")
            # créer des labels par défaut en cas d'échec
            labels_spectral = np.zeros(len(X_pca_scaled), dtype=int)
        
        # calculate metrics 
        for name, labels in [('KMeans', labels_km), ('Agglomerative', labels_agg), 
                           ('GMM', labels_gmm), ('Birch', labels_birch), ('Spectral', labels_spectral)]:
            if len(np.unique(labels)) > 1:  # Need at least 2 clusters for metrics (For 2D vizualisation)
                try:
                    metrics[name]['silhouette'].append(silhouette_score(X_pca_scaled, labels))
                    metrics[name]['calinski'].append(calinski_harabasz_score(X_pca_scaled, labels))
                    metrics[name]['davies'].append(davies_bouldin_score(X_pca_scaled, labels))
                except Exception as e:
                    print(f"   Error calculating metrics for {name} with {n_clusters} clusters: {e}")
                    metrics[name]['silhouette'].append(0)
                    metrics[name]['calinski'].append(0)
                    metrics[name]['davies'].append(float('inf'))
            else:
                metrics[name]['silhouette'].append(0)
                metrics[name]['calinski'].append(0)
                metrics[name]['davies'].append(float('inf'))

    # print optimal number of clusters
    print("Optimal number of clusters by metric:")
    for algorithm in metrics.keys():
        if metrics[algorithm]['silhouette']:
            try:
                best_sil = cluster_range[np.argmax(metrics[algorithm]['silhouette'])]
                best_cal = cluster_range[np.argmax(metrics[algorithm]['calinski'])]
                best_dav = cluster_range[np.argmin(metrics[algorithm]['davies'])]
                print(f"   {algorithm:>12}: Silhouette={best_sil}, Calinski={best_cal}, Davies-Bouldin={best_dav}")
            except Exception as e:
                print(f"   {algorithm:>12}: Error calculating optimal clusters - {e}")

    # Print the full score table per algorithm
    print("\n Full clustering scores by number of clusters:")
    for algorithm in metrics:
        print(f"\n {algorithm}")
        print("Clusters | Silhouette | Calinski-Harabasz | Davies-Bouldin")
        for i, n in enumerate(cluster_range):
            if i < len(metrics[algorithm]['silhouette']):
                sil = metrics[algorithm]['silhouette'][i]
                cal = metrics[algorithm]['calinski'][i]
                dav = metrics[algorithm]['davies'][i]
                print(f"{n:>8} | {sil:>10.3f} | {cal:>18.2f} | {dav:>15.3f}")
    
    clustering_results = {}

    # Fit each clustering algorithm with its chosen parameters on the PCA data
    # compute quality metrics (Silhouette, Calinski-Harabasz, Davies-Bouldin)
    # store the results and print the composition of each cluster.
    for method_name, params in clustering_methods.items():
        print(f"\n {method_name} Clustering")
        
        try:
            if method_name == "KMeans":
                model = KMeans(random_state=42, n_init=10, **params)
                labels = model.fit_predict(X_pca_scaled)
                
            elif method_name == "Agglomerative":
                model = AgglomerativeClustering(**params)
                labels = model.fit_predict(X_pca_scaled)
                
            elif method_name == "GMM":
                model = GaussianMixture(random_state=42, **params)
                labels = model.fit_predict(X_pca_scaled)
                
            elif method_name == "Spectral":
                model = SpectralClustering(random_state=42, affinity='nearest_neighbors', **params)
                labels = model.fit_predict(X_pca_scaled)
                
            elif method_name == "Birch":  
                model = Birch(**params)
                labels = model.fit_predict(X_pca_scaled)
            
            # evaluate clustering quality
            if len(np.unique(labels)) > 1:
                sil_score = silhouette_score(X_pca_scaled, labels)
                cal_score = calinski_harabasz_score(X_pca_scaled, labels)
                dav_score = davies_bouldin_score(X_pca_scaled, labels)
                
                print(f"   Silhouette Score: {sil_score:.3f}")
                print(f"   Calinski-Harabasz: {cal_score:.2f}")
                print(f"   Davies-Bouldin: {dav_score:.3f}")
                
                clustering_results[method_name] = {
                    'labels': labels,
                    'model': model,
                    'silhouette': sil_score,
                    'calinski': cal_score,
                    'davies_bouldin': dav_score
                }
                
                # Print cluster composition
                print(f"   Cluster composition:")
                for cluster_id in np.unique(labels):
                    cluster_teams = pca_df.index[labels == cluster_id].tolist()
                    print(f"      Cluster {cluster_id}: {cluster_teams}")
                    
        except Exception as e:
            print(f" Error with {method_name}: {e}")
    
    return clustering_results


def visualize_pca_clusters(pca_results, clustering_results):
    """
    Visualize clustering results on PCA space
    """
    print("\n CLUSTERING VISUALIZATION")
    print("-" * 50)

    #Extract PCA scores (teams and components) and variance ratios for labeling axes
    pca_df = pca_results['pca_df']
    explained_variance = pca_results['explained_variance_ratio']
    
    # create a visualization for each clustering method
    for method_name, result in clustering_results.items():
        labels = result['labels']
        sil_score = result['silhouette']
        
        # create plot dataframe
        plot_df = pca_df.copy()
        plot_df['Team'] = plot_df.index
        plot_df['Cluster'] = labels.astype(str)
        
        # 2D visualization for first two components
        if pca_df.shape[1] >= 2:
            fig = px.scatter(
                plot_df, x='Style_1', y='Style_2',
                color='Cluster', text='Team',
                category_orders={"Cluster": sorted(plot_df["Cluster"].unique(), key=lambda x: int(x))}, 
                title=f'{method_name} Clustering on Tactical Styles',
                labels={'Style_1': f'Style 1 ({explained_variance[0]:.1%})',
                       'Style_2': f'Style 2 ({explained_variance[1]:.1%})'},
                width=1000, height=600
            )
            fig.update_traces(textposition='top center')
            fig.show()
        
        # 3D visualization if we have at least 3 components
        if pca_df.shape[1] >= 3:
            fig = px.scatter_3d(
                plot_df, x='Style_1', y='Style_2', z='Style_3',
                color='Cluster', text='Team',
                category_orders={"Cluster": sorted(plot_df["Cluster"].unique(), key=lambda x: int(x))}, 
                title=f'{method_name} Clustering - 3D Tactical Space',
                labels={'Style_1': f'Style 1 ({explained_variance[0]:.1%})',
                       'Style_2': f'Style 2 ({explained_variance[1]:.1%})',
                       'Style_3': f'Style 3 ({explained_variance[2]:.1%})'},
                width=1000, height=700
            )
            fig.show()

def compute_global_discriminant_features(pca_results, clustering_results):
    """
    Compute average F-ratio (feature importance) across all clustering methods.
    """
    print("\n GLOBAL MOST DISCRIMINANT FEATURES (FOR ALL CLUSTERINGS)")
    print("=" * 60)

    # Prepare a dictionary to store F-ratios per feature for each clustering method
    original_data = pca_results['original_data']
    global_f_ratios = {feature: [] for feature in original_data.columns}

    # Loop over results from each clustering algorithm
    for method_name, result in clustering_results.items():
        labels = result['labels']
        original_with_clusters = original_data.copy()
        original_with_clusters['Cluster'] = labels
        unique_clusters = np.unique(labels)

        #Loop through every original metric (feature)
        for feature in original_data.columns:
            feature_values = original_with_clusters[feature].values
            overall_mean = np.mean(feature_values)

            between_ss = 0
            within_ss = 0

            # Loop through each cluster to accumulate between/within variance
            for cluster_id in unique_clusters:
                cluster_vals = original_with_clusters[original_with_clusters['Cluster'] == cluster_id][feature].values
                cluster_mean = np.mean(cluster_vals)
                between_ss += len(cluster_vals) * (cluster_mean - overall_mean) ** 2
                within_ss += np.sum((cluster_vals - cluster_mean) ** 2)

            # Compute the F-ratio: higher means the feature better separates clusters
            f_ratio = between_ss / within_ss if within_ss > 0 else 0
            # Save this F-ratio for the current feature and clustering method
            global_f_ratios[feature].append(f_ratio)

    # average F-ratios across algorithms
    avg_f_ratios = [(feature, np.mean(f_vals)) for feature, f_vals in global_f_ratios.items()]
    avg_f_ratios.sort(key=lambda x: x[1], reverse=True)

    print(" Averaged most discriminating features across clustering methods:")
    for i, (feature, avg_f) in enumerate(avg_f_ratios):
        print(f"   {i+1:2d}. {feature:<25} Avg F-ratio = {avg_f:.3f}")

    # main execution function
def run_pca_tactical_analysis(df_stocked):
    """
    Run a complete PCA-based tactical analysis
    """
    # 1: Apply PCA transformation
    pca_results = apply_pca_transformation(df_stocked, variance_threshold=0.85)
    
    # 2: Apply clustering to PCA components
    clustering_results = cluster_pca_styles(pca_results, max_clusters=9)
    
    # 3: Visualize clustering results
    visualize_pca_clusters(pca_results, clustering_results)

    # 4: Compute global discriminant features across all clusterings
    compute_global_discriminant_features(pca_results, clustering_results)

    #Return results for further analysis
    return {
        'pca_results': pca_results,
        'clustering_results': clustering_results
    }


# Run main function
results = run_pca_tactical_analysis(df_stocked)

#--------------------------------------------------------------------------------------------------------
#--------------------------------------------------------------------------------------------------------
#--------------------------------------------------------------------------------------------------------
# Additional functionalities if needed

# save the scores per team based on average with colors in images

# calculate average and categorize performance with 6 levels
metric_averages = df_stocked.mean()
performance = pd.DataFrame(index=df_stocked.index, columns=df_stocked.columns)

# Assign performance ratings for each team and metric using league average ± standard deviations
for col in df_stocked.columns:
    avg = metric_averages[col]
    std = df_stocked[col].std()
    for team in df_stocked.index:
        val = df_stocked.loc[team, col]
        if val >= avg + 2*std:
            cat = "+++"  # very good performance (2+ std above avg)
        elif val >= avg + std:
            cat = "++"   # good performance (1-2 std above avg)
        elif val >= avg:
            cat = "+"    # Above average performance
        elif val <= avg - 2*std:
            cat = "---"  # Very poor performance (2+ std below avg)
        elif val <= avg - std:
            cat = "--"   # Poor performance (1-2 std below avg)
        else:
            cat = "-"    # Below average performance
        performance.loc[team, col] = cat

# combine value and label
df_display = df_stocked.round(2).astype(str) + " (" + performance + ")"
df_display.loc["League Average"] = [f"{metric_averages[col]:.2f} (avg)" for col in df_stocked.columns]

# save Horizontal Table ----------
plt.figure(figsize=(len(df_display.columns) * 2.2, len(df_display) * 0.35))
sns.set(font_scale=0.72)
table = plt.table(cellText=df_display.values,
                  rowLabels=df_display.index,
                  colLabels=df_display.columns,
                  loc='center',
                  cellLoc='center')
table.auto_set_font_size(False)
table.set_fontsize(6.5)
table.scale(1.05, 1.08)

# add colors based on performance categories (6 levels)
for i in range(len(df_display)):
    for j in range(len(df_display.columns)):
        cell_text = df_display.iloc[i, j]
        if "(+++)" in cell_text:
            table[(i+1, j)].set_facecolor('#006400')  # Very dark green
            table[(i+1, j)].set_text_props(weight='bold', color='white')
        elif "(++)" in cell_text:
            table[(i+1, j)].set_facecolor('#2E8B57')  # Dark green
            table[(i+1, j)].set_text_props(weight='bold', color='white')
        elif "(+)" in cell_text:
            table[(i+1, j)].set_facecolor('#90EE90')  # Light green
            table[(i+1, j)].set_text_props(weight='bold')
        elif "(---)" in cell_text:
            table[(i+1, j)].set_facecolor('#8B0000')  # Very dark red
            table[(i+1, j)].set_text_props(weight='bold', color='white')
        elif "(--)" in cell_text:
            table[(i+1, j)].set_facecolor('#CD5C5C')  # Dark red
            table[(i+1, j)].set_text_props(weight='bold', color='white')
        elif "(-)" in cell_text:
            table[(i+1, j)].set_facecolor('#FFB6C1')  # Light red
            table[(i+1, j)].set_text_props(weight='bold')
        elif "(avg)" in cell_text:
            table[(i+1, j)].set_facecolor('#D3D3D3')  # Light gray
            table[(i+1, j)].set_text_props(weight='bold')

plt.title('Team Tactical Metrics - Horizontal View', fontsize=14, fontweight='bold', pad=20)
plt.axis('off')
plt.savefig("team_tactical_metrics_performance_table_PremierLeague_1516_EFA.png", bbox_inches='tight', dpi=300)


print("\nLegend:")
print("+++ (Very Dark Green): Exceptional - Above average + 2 standard deviations")
print("++ (Dark Green): Very Good - Above average + 1 standard deviation")
print("+ (Light Green): Good - Above average")
print("- (Light Red): Below average")
print("-- (Dark Red): Poor - Below average - 1 standard deviation")
print("--- (Very Dark Red): Very Poor - Below average - 2 standard deviations")
print("avg (Gray): League average")

# Display the categorized data 
print("\n Performance Categories:")
print(performance)

# Display average scores per metric
print("\n Average Scores per Metric:")
print(metric_averages.round(2))

## VAEP metrics only

In [ ]:
import joblib
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
import plotly.express as px
from socceraction.data.statsbomb import StatsBombLoader
from socceraction.spadl import statsbomb as spadl_sb
from socceraction import spadl
from socceraction.vaep import features as feat
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering, DBSCAN, Birch
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

#Load StatsBomb data locally
sbl = StatsBombLoader(
    getter="local",  # from local files
    root="C:/Users/helio/MachineLearning/Bachelor/statsbomb_data/open-data-master/data"
)

# Get all games for a given competition and season
# Competition ID = 2, Season ID = 27 (Premier League 2015/16)
# Competition ID = 11, Season ID = 27 (La Liga 2015/16)
# Competition ID = 9, Season ID = 27 (Bundesliga 2015/16)
# Competition ID = 12, Season ID = 27 (Serie A 2015/16)
# Competition ID = 7, Season ID = 27 (Ligue 1 2015/16)
games = sbl.games(2, 27)

# Load pre-trained VAEP model that some metrics need, especially VAEP metrics
data = joblib.load("C:/Users/helio/MachineLearning/Bachelor/vaep_model_xgboost_5leagues_1516.pkl")
# The trained model
model = data["model"]          
# Expected input features for model
expected_columns = data["feature_columns"] 


# Create a mapping of team ids to names for all matches of interest
matches_of_interest = games[["game_id", "home_team_id", "away_team_id"]].copy()
team_id_name_map = {}
for game_id in matches_of_interest["game_id"]:
    teams = sbl.teams(game_id)
    for _, row in teams.iterrows():
        team_id_name_map[row["team_id"]] = row["team_name"]

# Add team names to match dataframe
matches_of_interest["home_team_name"] = matches_of_interest["home_team_id"].map(team_id_name_map)
matches_of_interest["away_team_name"] = matches_of_interest["away_team_id"].map(team_id_name_map)

# Keep only rows with valid team names
matches_of_interest = matches_of_interest.dropna(subset=["home_team_name", "away_team_name"])


# compute VAEP-based team metrics for a single match
def evaluate_team_metrics(game_id, home_name, away_name, model):
    """
    Given a match ID and team names, computes various metrics
    """
    # Get match info
    match = games.loc[games["game_id"] == game_id].iloc[0]
    home_id, away_id = match["home_team_id"], match["away_team_id"]
    # Load raw events for the game
    events = sbl.events(game_id)
    # Convert StatsBomb events to SPADL format and add action names
    actions = spadl.add_names(spadl_sb.convert_to_actions(events, home_id))
    # Prepare output DataFrame indexed by team IDs
    df = pd.DataFrame(index=[home_id, away_id])

    # predict VAEP values for each action
    gamestates = feat.play_left_to_right(feat.gamestates(actions, nb_prev_actions=3), home_id)

    # Build feature matrix using multiple encoders
    X = pd.concat([f(gamestates) for f in [
        feat.actiontype_onehot, feat.result_onehot, feat.bodypart_onehot, feat.startlocation,
        feat.endlocation, feat.movement, feat.time
    ]], axis=1)

    # ensure all expected columns exist , fill missing ones with 0
    for col in expected_columns:
        if col not in X.columns:
            X[col] = 0
    X = X[expected_columns]
    # Predict VAEP for each action
    actions["vaep_pred"] = model.predict(X)


    # VAEP metrics
    df["VAEP_Passes"] = actions[actions["type_name"] == "pass"].groupby("team_id")["vaep_pred"].sum()
    df["VAEP_Crosses"] = actions[actions["type_name"].isin(["cross", "freekick_crossed", "corner_crossed"])].groupby("team_id")["vaep_pred"].sum()
    df["VAEP_Dribbles"] = actions[actions["type_name"] == "dribble"].groupby("team_id")["vaep_pred"].sum()
    df["VAEP_Shot"] = actions[actions["type_name"] == "shot"].groupby("team_id")["vaep_pred"].sum()
    df["VAEP_Clearance"] = actions[actions["type_name"] == "clearance"].groupby("team_id")["vaep_pred"].sum()
    df["VAEP_Interception"] = actions[actions["type_name"] == "interception"].groupby("team_id")["vaep_pred"].sum()
    df["VAEP_Foul"] = actions[actions["type_name"] == "foul"].groupby("team_id")["vaep_pred"].sum()
    df["VAEP_Tackle"] = actions[actions["type_name"] == "tackle"].groupby("team_id")["vaep_pred"].sum()
    df["VAEP_BadTouch"] = actions[actions["type_name"] == "bad_touch"].groupby("team_id")["vaep_pred"].sum()
    df["VAEP_KeeperSave"] = actions[actions["type_name"] == "keeper_save"].groupby("team_id")["vaep_pred"].sum()

    
    # fill missing values with 0
    df = df.fillna(0)
    # replace team ids with team names
    df.index = df.index.map(lambda x: home_name if x == home_id else away_name)
    return df

# store cumulative metrics for all teams
team_vaep_metrics = {}

# loop through each match
for _, row in tqdm(matches_of_interest.iterrows(), total=len(matches_of_interest)):
    game_id = row["game_id"]
    home_name = row["home_team_name"]
    away_name = row["away_team_name"]

    # get metrics for this match
    df = evaluate_team_metrics(game_id, home_name, away_name, model)

    # add metrics to each team (sum over matches)
    for team in df.index:
        if team not in team_vaep_metrics:
            team_vaep_metrics[team] = df.loc[team]  # first time just add
        else:
            team_vaep_metrics[team] += df.loc[team] # otherwise sum up

# convert dictionary to DataFrame for readability
df_stocked = pd.DataFrame(team_vaep_metrics).T.fillna(0)
print("Data available in df_stocked")
print(df_stocked)


def apply_pca_transformation(df_stocked, n_components=None, variance_threshold=0.95):
    """
    Apply PCA to create latent tactical style components
    
    """
    print(" PCA-BASED TACTICAL STYLE TRANSFORMATION")
    
    # prepare data
    X = df_stocked.copy()
    # make sure to have only one team and no duplicates
    X = X.loc[~X.index.duplicated(keep='first')]

    # CORRELATION ANALYSIS
    print("\n CORRELATION ANALYSIS")
    print("-" * 50)
    
    corr_matrix = X.corr()
    
    # find highly correlated features
    high_corr_pairs = []
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            if abs(corr_matrix.iloc[i, j]) > 0.7:
                high_corr_pairs.append((
                    corr_matrix.columns[i], 
                    corr_matrix.columns[j], 
                    corr_matrix.iloc[i, j]
                ))
    
    print(f"Highly correlated pairs (>0.7): {len(high_corr_pairs)}")
    for feat1, feat2, corr in high_corr_pairs:
        print(f"   {feat1} ↔ {feat2}: {corr:.3f}")
    
    # STANDARDIZATION
    print("\n DATA STANDARDIZATION")
    print("-" * 50)
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    print(" Data standardized (mean=0, std=1)")
    
    # PCA APPLICATION
    print("\n PCA APPLICATION")
    print("-" * 50)
    
    # apply PCA with all components first to analyze variance
    pca_full = PCA()
    X_pca_full = pca_full.fit_transform(X_scaled)
    
    # calculate cumulative variance
    added_variance = np.cumsum(pca_full.explained_variance_ratio_)
    
    # determine the number of components
    if n_components is None:
        n_components = np.argmax(added_variance >= variance_threshold) + 1
        print(f"Components needed for {variance_threshold*100}% variance: {n_components}")
    
    # apply PCA with the chosen number of components
    pca = PCA(n_components=n_components, random_state=42)
    X_pca = pca.fit_transform(X_scaled)
    
    print(f"Final PCA shape: {X_pca.shape}")
    print(f"Explained variance ratio: {pca.explained_variance_ratio_}")
    print(f"Cumulative variance explained: {np.sum(pca.explained_variance_ratio_):.3f}")
    
    # component interpretation
    print("\n TACTICAL STYLE COMPONENTS INTERPRETATION")
    print("-" * 50)
    
    # create component interpretation
    components_df = pd.DataFrame(
        pca.components_.T,
        columns=[f'Style_{i+1}' for i in range(n_components)],
        index=X.columns
    )
    
    # interpret each component
    tactical_styles = {}
    for i in range(n_components):
        component_name = f'Style_{i+1}'
        component_loadings = components_df[component_name].abs().sort_values(ascending=False)
        
        print(f"\n {component_name} (explains {pca.explained_variance_ratio_[i]:.1%} variance):")
        
        # top positive loadings
        pos_loadings = components_df[component_name].sort_values(ascending=False).head(3)
        print("   Top positive loadings:")
        for feat, loading in pos_loadings.items():
            print(f"      +{loading:.3f} {feat}")
        
        # top negative loadings
        neg_loadings = components_df[component_name].sort_values(ascending=True).head(3)
        print("   Top negative loadings:")
        for feat, loading in neg_loadings.items():
            print(f"      {loading:.3f} {feat}")
        
        # suggest a tactical interpretation
        top_features = component_loadings.head(3).index.tolist()
        tactical_styles[component_name] = {
            'variance_explained': pca.explained_variance_ratio_[i],
            'top_features': top_features
        }
    
    # Create a PCA dataFrame for visibility and better understandings
    pca_df = pd.DataFrame(
        X_pca, 
        columns=[f'Style_{i+1}' for i in range(n_components)],
        index=X.index
    )
    
    return {
        'pca_df': pca_df,
        'pca_model': pca,
        'scaler': scaler,
        'components_df': components_df,
        'tactical_styles': tactical_styles,
        'original_data': X,
        'scaled_data': X_scaled,
        'explained_variance_ratio': pca.explained_variance_ratio_,
        'added_variance': np.cumsum(pca.explained_variance_ratio_)
    }

def cluster_pca_styles(pca_results, clustering_methods=None, max_clusters=10):
    """
    Apply clustering to PCA-transformed tactical styles
    """
    # The 5 clustering algorithms
    if clustering_methods is None:
        clustering_methods = {
            "KMeans": {"n_clusters": 9},
            "Agglomerative": {"n_clusters": 9},
            "GMM": {"n_components": 9},
            "Spectral": {"n_clusters": 9},
            "Birch": {"n_clusters": 6}
        }
    

    # Extract the PCA-transformed feature matrix (scores) from results
    # Get PCA scores as a NumPy array for clustering
    pca_df = pca_results['pca_df']
    X_pca = pca_df.values
    
    # standardize PCA components
    scaler_pca = StandardScaler()
    X_pca_scaled = scaler_pca.fit_transform(X_pca)


    print("\n OPTIMAL NUMBER OF CLUSTERS")
    print("-" * 50)

    # Test different numbers of clusters
    cluster_range = range(2, max_clusters + 1)

    # metrics for different clustering algorithms
    metrics = {
        'KMeans': {'silhouette': [], 'calinski': [], 'davies': []},
        'Agglomerative': {'silhouette': [], 'calinski': [], 'davies': []},
        'GMM': {'silhouette': [], 'calinski': [], 'davies': []},
        'Birch': {'silhouette': [], 'calinski': [], 'davies': []},
        'Spectral': {'silhouette': [], 'calinski': [], 'davies': []}
    }

    # Try different numbers of clusters for each algorithm and record quality metrics (Silhouette, Calinski-Harabasz, Davies-Bouldin)
    for n_clusters in cluster_range:
        
        # KMeans
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        labels_km = kmeans.fit_predict(X_pca_scaled)
        
        # Agglomerative
        agg = AgglomerativeClustering(n_clusters=n_clusters)
        labels_agg = agg.fit_predict(X_pca_scaled)
        
        # GMM
        gmm = GaussianMixture(n_components=n_clusters, random_state=42)
        labels_gmm = gmm.fit_predict(X_pca_scaled)
        
        # Birch 
        birch = Birch(n_clusters=n_clusters)
        labels_birch = birch.fit_predict(X_pca_scaled)
        
        # Spectral
        try:
            spectral = SpectralClustering(n_clusters=n_clusters, random_state=42, affinity='nearest_neighbors')
            labels_spectral = spectral.fit_predict(X_pca_scaled)
        except Exception as e:
            print(f"  Spectral clustering failed for {n_clusters} clusters: {e}")
            # créer des labels par défaut en cas d'échec
            labels_spectral = np.zeros(len(X_pca_scaled), dtype=int)
        
        # calculate metrics 
        for name, labels in [('KMeans', labels_km), ('Agglomerative', labels_agg), 
                           ('GMM', labels_gmm), ('Birch', labels_birch), ('Spectral', labels_spectral)]:
            if len(np.unique(labels)) > 1:  # Need at least 2 clusters for metrics (For 2D vizualisation)
                try:
                    metrics[name]['silhouette'].append(silhouette_score(X_pca_scaled, labels))
                    metrics[name]['calinski'].append(calinski_harabasz_score(X_pca_scaled, labels))
                    metrics[name]['davies'].append(davies_bouldin_score(X_pca_scaled, labels))
                except Exception as e:
                    print(f"   Error calculating metrics for {name} with {n_clusters} clusters: {e}")
                    metrics[name]['silhouette'].append(0)
                    metrics[name]['calinski'].append(0)
                    metrics[name]['davies'].append(float('inf'))
            else:
                metrics[name]['silhouette'].append(0)
                metrics[name]['calinski'].append(0)
                metrics[name]['davies'].append(float('inf'))

    # print optimal number of clusters
    print("Optimal number of clusters by metric:")
    for algorithm in metrics.keys():
        if metrics[algorithm]['silhouette']:
            try:
                best_sil = cluster_range[np.argmax(metrics[algorithm]['silhouette'])]
                best_cal = cluster_range[np.argmax(metrics[algorithm]['calinski'])]
                best_dav = cluster_range[np.argmin(metrics[algorithm]['davies'])]
                print(f"   {algorithm:>12}: Silhouette={best_sil}, Calinski={best_cal}, Davies-Bouldin={best_dav}")
            except Exception as e:
                print(f"   {algorithm:>12}: Error calculating optimal clusters - {e}")

    # Print the full score table per algorithm
    print("\n Full clustering scores by number of clusters:")
    for algorithm in metrics:
        print(f"\n {algorithm}")
        print("Clusters | Silhouette | Calinski-Harabasz | Davies-Bouldin")
        for i, n in enumerate(cluster_range):
            if i < len(metrics[algorithm]['silhouette']):
                sil = metrics[algorithm]['silhouette'][i]
                cal = metrics[algorithm]['calinski'][i]
                dav = metrics[algorithm]['davies'][i]
                print(f"{n:>8} | {sil:>10.3f} | {cal:>18.2f} | {dav:>15.3f}")
    
    clustering_results = {}

    # Fit each clustering algorithm with its chosen parameters on the PCA data
    # compute quality metrics (Silhouette, Calinski-Harabasz, Davies-Bouldin)
    # store the results and print the composition of each cluster.
    for method_name, params in clustering_methods.items():
        print(f"\n {method_name} Clustering")
        
        try:
            if method_name == "KMeans":
                model = KMeans(random_state=42, n_init=10, **params)
                labels = model.fit_predict(X_pca_scaled)
                
            elif method_name == "Agglomerative":
                model = AgglomerativeClustering(**params)
                labels = model.fit_predict(X_pca_scaled)
                
            elif method_name == "GMM":
                model = GaussianMixture(random_state=42, **params)
                labels = model.fit_predict(X_pca_scaled)
                
            elif method_name == "Spectral":
                model = SpectralClustering(random_state=42, affinity='nearest_neighbors', **params)
                labels = model.fit_predict(X_pca_scaled)
                
            elif method_name == "Birch":  
                model = Birch(**params)
                labels = model.fit_predict(X_pca_scaled)
            
            # evaluate clustering quality
            if len(np.unique(labels)) > 1:
                sil_score = silhouette_score(X_pca_scaled, labels)
                cal_score = calinski_harabasz_score(X_pca_scaled, labels)
                dav_score = davies_bouldin_score(X_pca_scaled, labels)
                
                print(f"   Silhouette Score: {sil_score:.3f}")
                print(f"   Calinski-Harabasz: {cal_score:.2f}")
                print(f"   Davies-Bouldin: {dav_score:.3f}")
                
                clustering_results[method_name] = {
                    'labels': labels,
                    'model': model,
                    'silhouette': sil_score,
                    'calinski': cal_score,
                    'davies_bouldin': dav_score
                }
                
                # Print cluster composition
                print(f"   Cluster composition:")
                for cluster_id in np.unique(labels):
                    cluster_teams = pca_df.index[labels == cluster_id].tolist()
                    print(f"      Cluster {cluster_id}: {cluster_teams}")
                    
        except Exception as e:
            print(f" Error with {method_name}: {e}")
    
    return clustering_results


def visualize_pca_clusters(pca_results, clustering_results):
    """
    Visualize clustering results on PCA space
    """
    print("\n CLUSTERING VISUALIZATION")
    print("-" * 50)

    #Extract PCA scores (teams and components) and variance ratios for labeling axes
    pca_df = pca_results['pca_df']
    explained_variance = pca_results['explained_variance_ratio']
    
    # create a visualization for each clustering method
    for method_name, result in clustering_results.items():
        labels = result['labels']
        sil_score = result['silhouette']
        
        # create plot dataframe
        plot_df = pca_df.copy()
        plot_df['Team'] = plot_df.index
        plot_df['Cluster'] = labels.astype(str)
        
        # 2D visualization for first two components
        if pca_df.shape[1] >= 2:
            fig = px.scatter(
                plot_df, x='Style_1', y='Style_2',
                color='Cluster', text='Team',
                category_orders={"Cluster": sorted(plot_df["Cluster"].unique(), key=lambda x: int(x))}, 
                title=f'{method_name} Clustering on Tactical Styles',
                labels={'Style_1': f'Style 1 ({explained_variance[0]:.1%})',
                       'Style_2': f'Style 2 ({explained_variance[1]:.1%})'},
                width=1000, height=600
            )
            fig.update_traces(textposition='top center')
            fig.show()
        
        # 3D visualization if we have at least 3 components
        if pca_df.shape[1] >= 3:
            fig = px.scatter_3d(
                plot_df, x='Style_1', y='Style_2', z='Style_3',
                color='Cluster', text='Team',
                category_orders={"Cluster": sorted(plot_df["Cluster"].unique(), key=lambda x: int(x))}, 
                title=f'{method_name} Clustering - 3D Tactical Space',
                labels={'Style_1': f'Style 1 ({explained_variance[0]:.1%})',
                       'Style_2': f'Style 2 ({explained_variance[1]:.1%})',
                       'Style_3': f'Style 3 ({explained_variance[2]:.1%})'},
                width=1000, height=700
            )
            fig.show()

def compute_global_discriminant_features(pca_results, clustering_results):
    """
    Compute average F-ratio (feature importance) across all clustering methods.
    """
    print("\n GLOBAL MOST DISCRIMINANT FEATURES (FOR ALL CLUSTERINGS)")
    print("=" * 60)

    # Prepare a dictionary to store F-ratios per feature for each clustering method
    original_data = pca_results['original_data']
    global_f_ratios = {feature: [] for feature in original_data.columns}

    # Loop over results from each clustering algorithm
    for method_name, result in clustering_results.items():
        labels = result['labels']
        original_with_clusters = original_data.copy()
        original_with_clusters['Cluster'] = labels
        unique_clusters = np.unique(labels)

        #Loop through every original metric (feature)
        for feature in original_data.columns:
            feature_values = original_with_clusters[feature].values
            overall_mean = np.mean(feature_values)

            between_ss = 0
            within_ss = 0

            # Loop through each cluster to accumulate between/within variance
            for cluster_id in unique_clusters:
                cluster_vals = original_with_clusters[original_with_clusters['Cluster'] == cluster_id][feature].values
                cluster_mean = np.mean(cluster_vals)
                between_ss += len(cluster_vals) * (cluster_mean - overall_mean) ** 2
                within_ss += np.sum((cluster_vals - cluster_mean) ** 2)

            # Compute the F-ratio: higher means the feature better separates clusters
            f_ratio = between_ss / within_ss if within_ss > 0 else 0
            # Save this F-ratio for the current feature and clustering method
            global_f_ratios[feature].append(f_ratio)

    # average F-ratios across algorithms
    avg_f_ratios = [(feature, np.mean(f_vals)) for feature, f_vals in global_f_ratios.items()]
    avg_f_ratios.sort(key=lambda x: x[1], reverse=True)

    print(" Averaged most discriminating features across clustering methods:")
    for i, (feature, avg_f) in enumerate(avg_f_ratios):
        print(f"   {i+1:2d}. {feature:<25} Avg F-ratio = {avg_f:.3f}")

    # main execution function
def run_pca_tactical_analysis(df_stocked):
    """
    Run a complete PCA-based tactical analysis
    """
    # 1: Apply PCA transformation
    pca_results = apply_pca_transformation(df_stocked, variance_threshold=0.85)
    
    # 2: Apply clustering to PCA components
    clustering_results = cluster_pca_styles(pca_results, max_clusters=9)
    
    # 3: Visualize clustering results
    visualize_pca_clusters(pca_results, clustering_results)

    # 4: Compute global discriminant features across all clusterings
    compute_global_discriminant_features(pca_results, clustering_results)

    #Return results for further analysis
    return {
        'pca_results': pca_results,
        'clustering_results': clustering_results
    }


# Run main function
results = run_pca_tactical_analysis(df_stocked)

#--------------------------------------------------------------------------------------------------------
#--------------------------------------------------------------------------------------------------------
#--------------------------------------------------------------------------------------------------------
# Additional functionalities if needed

# save the scores per team based on average with colors in images

# calculate average and categorize performance with 6 levels
metric_averages = df_stocked.mean()
performance = pd.DataFrame(index=df_stocked.index, columns=df_stocked.columns)

# Assign performance ratings for each team and metric using league average ± standard deviations
for col in df_stocked.columns:
    avg = metric_averages[col]
    std = df_stocked[col].std()
    for team in df_stocked.index:
        val = df_stocked.loc[team, col]
        if val >= avg + 2*std:
            cat = "+++"  # very good performance (2+ std above avg)
        elif val >= avg + std:
            cat = "++"   # good performance (1-2 std above avg)
        elif val >= avg:
            cat = "+"    # Above average performance
        elif val <= avg - 2*std:
            cat = "---"  # Very poor performance (2+ std below avg)
        elif val <= avg - std:
            cat = "--"   # Poor performance (1-2 std below avg)
        else:
            cat = "-"    # Below average performance
        performance.loc[team, col] = cat

# combine value and label
df_display = df_stocked.round(2).astype(str) + " (" + performance + ")"
df_display.loc["League Average"] = [f"{metric_averages[col]:.2f} (avg)" for col in df_stocked.columns]

# save Horizontal Table ----------
plt.figure(figsize=(len(df_display.columns) * 2.2, len(df_display) * 0.35))
sns.set(font_scale=0.72)
table = plt.table(cellText=df_display.values,
                  rowLabels=df_display.index,
                  colLabels=df_display.columns,
                  loc='center',
                  cellLoc='center')
table.auto_set_font_size(False)
table.set_fontsize(6.5)
table.scale(1.05, 1.08)

# add colors based on performance categories (6 levels)
for i in range(len(df_display)):
    for j in range(len(df_display.columns)):
        cell_text = df_display.iloc[i, j]
        if "(+++)" in cell_text:
            table[(i+1, j)].set_facecolor('#006400')  # Very dark green
            table[(i+1, j)].set_text_props(weight='bold', color='white')
        elif "(++)" in cell_text:
            table[(i+1, j)].set_facecolor('#2E8B57')  # Dark green
            table[(i+1, j)].set_text_props(weight='bold', color='white')
        elif "(+)" in cell_text:
            table[(i+1, j)].set_facecolor('#90EE90')  # Light green
            table[(i+1, j)].set_text_props(weight='bold')
        elif "(---)" in cell_text:
            table[(i+1, j)].set_facecolor('#8B0000')  # Very dark red
            table[(i+1, j)].set_text_props(weight='bold', color='white')
        elif "(--)" in cell_text:
            table[(i+1, j)].set_facecolor('#CD5C5C')  # Dark red
            table[(i+1, j)].set_text_props(weight='bold', color='white')
        elif "(-)" in cell_text:
            table[(i+1, j)].set_facecolor('#FFB6C1')  # Light red
            table[(i+1, j)].set_text_props(weight='bold')
        elif "(avg)" in cell_text:
            table[(i+1, j)].set_facecolor('#D3D3D3')  # Light gray
            table[(i+1, j)].set_text_props(weight='bold')

plt.title('Team Tactical Metrics - Horizontal View', fontsize=14, fontweight='bold', pad=20)
plt.axis('off')
plt.savefig("team_tactical_metrics_performance_table_PremierLeague_1516_VAPEONLY.png", bbox_inches='tight', dpi=300)


print("\nLegend:")
print("+++ (Very Dark Green): Exceptional - Above average + 2 standard deviations")
print("++ (Dark Green): Very Good - Above average + 1 standard deviation")
print("+ (Light Green): Good - Above average")
print("- (Light Red): Below average")
print("-- (Dark Red): Poor - Below average - 1 standard deviation")
print("--- (Very Dark Red): Very Poor - Below average - 2 standard deviations")
print("avg (Gray): League average")

# Display the categorized data 
print("\n Performance Categories:")
print(performance)

# Display average scores per metric
print("\n Average Scores per Metric:")
print(metric_averages.round(2))

## VAEP + other metrics

In [ ]:
import joblib
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
import plotly.express as px
from socceraction.data.statsbomb import StatsBombLoader
from socceraction.spadl import statsbomb as spadl_sb
from socceraction import spadl
from socceraction.vaep import features as feat
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering, DBSCAN, Birch
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score


#Load StatsBomb data locally
sbl = StatsBombLoader(
    getter="local",  # from local files
    root="C:/Users/helio/MachineLearning/Bachelor/statsbomb_data/open-data-master/data"
)

# Get all games for a given competition and season
# Competition ID = 2, Season ID = 27 (Premier League 2015/16)
# Competition ID = 11, Season ID = 27 (La Liga 2015/16)
# Competition ID = 9, Season ID = 27 (Bundesliga 2015/16)
# Competition ID = 12, Season ID = 27 (Serie A 2015/16)
# Competition ID = 7, Season ID = 27 (Ligue 1 2015/16)
games = sbl.games(2, 27)

# Load pre-trained VAEP model that some metrics need, especially VAEP metrics
data = joblib.load("C:/Users/helio/MachineLearning/Bachelor/vaep_model_xgboost_5leagues_1516.pkl")
# The trained model
model = data["model"]          
# Expected input features for model
expected_columns = data["feature_columns"] 


# Create a mapping of team ids to names for all matches of interest
matches_of_interest = games[["game_id", "home_team_id", "away_team_id"]].copy()
team_id_name_map = {}
for game_id in matches_of_interest["game_id"]:
    teams = sbl.teams(game_id)
    for _, row in teams.iterrows():
        team_id_name_map[row["team_id"]] = row["team_name"]

# Add team names to match dataframe
matches_of_interest["home_team_name"] = matches_of_interest["home_team_id"].map(team_id_name_map)
matches_of_interest["away_team_name"] = matches_of_interest["away_team_id"].map(team_id_name_map)

# Keep only rows with valid team names
matches_of_interest = matches_of_interest.dropna(subset=["home_team_name", "away_team_name"])


# compute VAEP-based team metrics for a single match
def evaluate_team_metrics(game_id, home_name, away_name, model):
    """
    Given a match ID and team names, computes various metrics
    """
    # Get match info
    match = games.loc[games["game_id"] == game_id].iloc[0]
    home_id, away_id = match["home_team_id"], match["away_team_id"]

    # Load raw events for the game
    events = sbl.events(game_id)

    # Convert StatsBomb events to SPADL format and add action names
    actions = spadl.add_names(spadl_sb.convert_to_actions(events, home_id))

    # Prepare output DataFrame indexed by team IDs
    df = pd.DataFrame(index=[home_id, away_id])

    # predict VAEP values for each action
    gamestates = feat.play_left_to_right(feat.gamestates(actions, nb_prev_actions=3), home_id)

    # Build feature matrix using multiple encoders
    X = pd.concat([f(gamestates) for f in [
        feat.actiontype_onehot, feat.result_onehot, feat.bodypart_onehot, feat.startlocation,
        feat.endlocation, feat.movement, feat.time
    ]], axis=1)

    # ensure all expected columns exist , fill missing ones with 0
    for col in expected_columns:
        if col not in X.columns:
            X[col] = 0
    X = X[expected_columns]

    # Predict VAEP for each action
    actions["vaep_pred"] = model.predict(X)

    # Events enrichements , adding basic info to each action
    # time since previous action
    actions["time_diff"] = actions["time_seconds"].diff().fillna(0)
    # movement distance of the ball
    actions["distance"] = np.sqrt((actions['end_x'] - actions['start_x'])**2 + (actions['end_y'] - actions['start_y'])**2)

     # VAEP metrics
    df["VAEP_Passes"] = actions[actions["type_name"] == "pass"].groupby("team_id")["vaep_pred"].sum()
    df["VAEP_Crosses"] = actions[actions["type_name"].isin(["cross", "freekick_crossed", "corner_crossed"])].groupby("team_id")["vaep_pred"].sum()
    df["VAEP_Dribbles"] = actions[actions["type_name"] == "dribble"].groupby("team_id")["vaep_pred"].sum()
    df["VAEP_Shot"] = actions[actions["type_name"] == "shot"].groupby("team_id")["vaep_pred"].sum()
    df["VAEP_Clearance"] = actions[actions["type_name"] == "clearance"].groupby("team_id")["vaep_pred"].sum()
    df["VAEP_Interception"] = actions[actions["type_name"] == "interception"].groupby("team_id")["vaep_pred"].sum()
    df["VAEP_Foul"] = actions[actions["type_name"] == "foul"].groupby("team_id")["vaep_pred"].sum()
    df["VAEP_Tackle"] = actions[actions["type_name"] == "tackle"].groupby("team_id")["vaep_pred"].sum()
    df["VAEP_BadTouch"] = actions[actions["type_name"] == "bad_touch"].groupby("team_id")["vaep_pred"].sum()
    df["VAEP_KeeperSave"] = actions[actions["type_name"] == "keeper_save"].groupby("team_id")["vaep_pred"].sum()


    # Other metrics , traditional metrics and not VAEP based
    passes = actions[actions['type_name'] == 'pass']
    defensive_actions = actions[actions['type_name'].isin(['tackle', 'interception', 'clearance']) & (actions['start_x'] >= 40)]
    ppda_n = passes.groupby('team_id').size()
    ppda_d = defensive_actions.groupby('team_id').size()
    df['PPDA'] = ppda_n / ppda_d

    progressive_passes = passes.copy()
    total_distance_pass = np.sqrt((progressive_passes['end_x'] - progressive_passes['start_x'])**2 + (progressive_passes['end_y'] - progressive_passes['start_y'])**2)
    vertical_distance = progressive_passes['end_x'] - progressive_passes['start_x']
    verticality_ratio = vertical_distance / total_distance_pass.replace(0, np.nan)
    df['Verticality'] = progressive_passes.assign(ratio=verticality_ratio).groupby('team_id')['ratio'].mean()
    df['ProgressivePasses'] = progressive_passes[progressive_passes['end_x'] - progressive_passes['start_x'] > 10].groupby('team_id').size()
    df['BuildUpSpeed'] = actions.groupby('team_id')['distance'].sum() / actions.groupby('team_id')['time_diff'].sum()

    # fill missing values with 0
    df = df.fillna(0)
    # replace team ids with team names
    df.index = df.index.map(lambda x: home_name if x == home_id else away_name)
    return df

# store cumulative metrics for all teams
team_vaep_metrics = {}

# loop through each match
for _, row in tqdm(matches_of_interest.iterrows(), total=len(matches_of_interest)):
    game_id = row["game_id"]
    home_name = row["home_team_name"]
    away_name = row["away_team_name"]

    # get metrics for this match
    df = evaluate_team_metrics(game_id, home_name, away_name, model)

    # add metrics to each team (sum over matches)
    for team in df.index:
        if team not in team_vaep_metrics:
            team_vaep_metrics[team] = df.loc[team]  # first time just add
        else:
            team_vaep_metrics[team] += df.loc[team] # otherwise sum up

# convert dictionary to DataFrame for readability
df_stocked = pd.DataFrame(team_vaep_metrics).T.fillna(0)
print("Data available in df_stocked")
print(df_stocked)


def apply_pca_transformation(df_stocked, n_components=None, variance_threshold=0.95):
    """
    Apply PCA to create latent tactical style components
    
    """
    print(" PCA-BASED TACTICAL STYLE TRANSFORMATION")
    
    # prepare data
    X = df_stocked.copy()
    # make sure to have only one team and no duplicates
    X = X.loc[~X.index.duplicated(keep='first')]

    # CORRELATION ANALYSIS
    print("\n CORRELATION ANALYSIS")
    print("-" * 50)
    
    corr_matrix = X.corr()
    
    # find highly correlated features
    high_corr_pairs = []
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            if abs(corr_matrix.iloc[i, j]) > 0.7:
                high_corr_pairs.append((
                    corr_matrix.columns[i], 
                    corr_matrix.columns[j], 
                    corr_matrix.iloc[i, j]
                ))
    
    print(f"Highly correlated pairs (>0.7): {len(high_corr_pairs)}")
    for feat1, feat2, corr in high_corr_pairs:
        print(f"   {feat1} ↔ {feat2}: {corr:.3f}")
    
    # STANDARDIZATION
    print("\n DATA STANDARDIZATION")
    print("-" * 50)
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    print(" Data standardized (mean=0, std=1)")
    
    # PCA APPLICATION
    print("\n PCA APPLICATION")
    print("-" * 50)
    
    # apply PCA with all components first to analyze variance
    pca_full = PCA()
    X_pca_full = pca_full.fit_transform(X_scaled)
    
    # calculate cumulative variance
    added_variance = np.cumsum(pca_full.explained_variance_ratio_)
    
    # determine the number of components
    if n_components is None:
        n_components = np.argmax(added_variance >= variance_threshold) + 1
        print(f"Components needed for {variance_threshold*100}% variance: {n_components}")
    
    # apply PCA with the chosen number of components
    pca = PCA(n_components=n_components, random_state=42)
    X_pca = pca.fit_transform(X_scaled)
    
    print(f"Final PCA shape: {X_pca.shape}")
    print(f"Explained variance ratio: {pca.explained_variance_ratio_}")
    print(f"Cumulative variance explained: {np.sum(pca.explained_variance_ratio_):.3f}")
    
    # component interpretation
    print("\n TACTICAL STYLE COMPONENTS INTERPRETATION")
    print("-" * 50)
    
    # create component interpretation
    components_df = pd.DataFrame(
        pca.components_.T,
        columns=[f'Style_{i+1}' for i in range(n_components)],
        index=X.columns
    )
    
    # interpret each component
    tactical_styles = {}
    for i in range(n_components):
        component_name = f'Style_{i+1}'
        component_loadings = components_df[component_name].abs().sort_values(ascending=False)
        
        print(f"\n {component_name} (explains {pca.explained_variance_ratio_[i]:.1%} variance):")
        
        # top positive loadings
        pos_loadings = components_df[component_name].sort_values(ascending=False).head(3)
        print("   Top positive loadings:")
        for feat, loading in pos_loadings.items():
            print(f"      +{loading:.3f} {feat}")
        
        # top negative loadings
        neg_loadings = components_df[component_name].sort_values(ascending=True).head(3)
        print("   Top negative loadings:")
        for feat, loading in neg_loadings.items():
            print(f"      {loading:.3f} {feat}")
        
        # suggest a tactical interpretation
        top_features = component_loadings.head(3).index.tolist()
        tactical_styles[component_name] = {
            'variance_explained': pca.explained_variance_ratio_[i],
            'top_features': top_features
        }
    
    # Create a PCA dataFrame for visibility and better understandings
    pca_df = pd.DataFrame(
        X_pca, 
        columns=[f'Style_{i+1}' for i in range(n_components)],
        index=X.index
    )
    
    return {
        'pca_df': pca_df,
        'pca_model': pca,
        'scaler': scaler,
        'components_df': components_df,
        'tactical_styles': tactical_styles,
        'original_data': X,
        'scaled_data': X_scaled,
        'explained_variance_ratio': pca.explained_variance_ratio_,
        'added_variance': np.cumsum(pca.explained_variance_ratio_)
    }

def cluster_pca_styles(pca_results, clustering_methods=None, max_clusters=10):
    """
    Apply clustering to PCA-transformed tactical styles
    """
    # The 5 clustering algorithms
    if clustering_methods is None:
        clustering_methods = {
            "KMeans": {"n_clusters": 9},
            "Agglomerative": {"n_clusters": 9},
            "GMM": {"n_components": 9},
            "Spectral": {"n_clusters": 9},
            "Birch": {"n_clusters": 6}
        }
    

    # Extract the PCA-transformed feature matrix (scores) from results
    # Get PCA scores as a NumPy array for clustering
    pca_df = pca_results['pca_df']
    X_pca = pca_df.values
    
    # standardize PCA components
    scaler_pca = StandardScaler()
    X_pca_scaled = scaler_pca.fit_transform(X_pca)


    print("\n OPTIMAL NUMBER OF CLUSTERS")
    print("-" * 50)

    # Test different numbers of clusters
    cluster_range = range(2, max_clusters + 1)

    # metrics for different clustering algorithms
    metrics = {
        'KMeans': {'silhouette': [], 'calinski': [], 'davies': []},
        'Agglomerative': {'silhouette': [], 'calinski': [], 'davies': []},
        'GMM': {'silhouette': [], 'calinski': [], 'davies': []},
        'Birch': {'silhouette': [], 'calinski': [], 'davies': []},
        'Spectral': {'silhouette': [], 'calinski': [], 'davies': []}
    }

    # Try different numbers of clusters for each algorithm and record quality metrics (Silhouette, Calinski-Harabasz, Davies-Bouldin)
    for n_clusters in cluster_range:
        
        # KMeans
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        labels_km = kmeans.fit_predict(X_pca_scaled)
        
        # Agglomerative
        agg = AgglomerativeClustering(n_clusters=n_clusters)
        labels_agg = agg.fit_predict(X_pca_scaled)
        
        # GMM
        gmm = GaussianMixture(n_components=n_clusters, random_state=42)
        labels_gmm = gmm.fit_predict(X_pca_scaled)
        
        # Birch 
        birch = Birch(n_clusters=n_clusters)
        labels_birch = birch.fit_predict(X_pca_scaled)
        
        # Spectral
        try:
            spectral = SpectralClustering(n_clusters=n_clusters, random_state=42, affinity='nearest_neighbors')
            labels_spectral = spectral.fit_predict(X_pca_scaled)
        except Exception as e:
            print(f"  Spectral clustering failed for {n_clusters} clusters: {e}")
            # créer des labels par défaut en cas d'échec
            labels_spectral = np.zeros(len(X_pca_scaled), dtype=int)
        
        # calculate metrics 
        for name, labels in [('KMeans', labels_km), ('Agglomerative', labels_agg), 
                           ('GMM', labels_gmm), ('Birch', labels_birch), ('Spectral', labels_spectral)]:
            if len(np.unique(labels)) > 1:  # Need at least 2 clusters for metrics (For 2D vizualisation)
                try:
                    metrics[name]['silhouette'].append(silhouette_score(X_pca_scaled, labels))
                    metrics[name]['calinski'].append(calinski_harabasz_score(X_pca_scaled, labels))
                    metrics[name]['davies'].append(davies_bouldin_score(X_pca_scaled, labels))
                except Exception as e:
                    print(f"   Error calculating metrics for {name} with {n_clusters} clusters: {e}")
                    metrics[name]['silhouette'].append(0)
                    metrics[name]['calinski'].append(0)
                    metrics[name]['davies'].append(float('inf'))
            else:
                metrics[name]['silhouette'].append(0)
                metrics[name]['calinski'].append(0)
                metrics[name]['davies'].append(float('inf'))

    # print optimal number of clusters
    print("Optimal number of clusters by metric:")
    for algorithm in metrics.keys():
        if metrics[algorithm]['silhouette']:
            try:
                best_sil = cluster_range[np.argmax(metrics[algorithm]['silhouette'])]
                best_cal = cluster_range[np.argmax(metrics[algorithm]['calinski'])]
                best_dav = cluster_range[np.argmin(metrics[algorithm]['davies'])]
                print(f"   {algorithm:>12}: Silhouette={best_sil}, Calinski={best_cal}, Davies-Bouldin={best_dav}")
            except Exception as e:
                print(f"   {algorithm:>12}: Error calculating optimal clusters - {e}")

    # Print the full score table per algorithm
    print("\n Full clustering scores by number of clusters:")
    for algorithm in metrics:
        print(f"\n {algorithm}")
        print("Clusters | Silhouette | Calinski-Harabasz | Davies-Bouldin")
        for i, n in enumerate(cluster_range):
            if i < len(metrics[algorithm]['silhouette']):
                sil = metrics[algorithm]['silhouette'][i]
                cal = metrics[algorithm]['calinski'][i]
                dav = metrics[algorithm]['davies'][i]
                print(f"{n:>8} | {sil:>10.3f} | {cal:>18.2f} | {dav:>15.3f}")
    
    clustering_results = {}

    # Fit each clustering algorithm with its chosen parameters on the PCA data
    # compute quality metrics (Silhouette, Calinski-Harabasz, Davies-Bouldin)
    # store the results and print the composition of each cluster.
    for method_name, params in clustering_methods.items():
        print(f"\n {method_name} Clustering")
        
        try:
            if method_name == "KMeans":
                model = KMeans(random_state=42, n_init=10, **params)
                labels = model.fit_predict(X_pca_scaled)
                
            elif method_name == "Agglomerative":
                model = AgglomerativeClustering(**params)
                labels = model.fit_predict(X_pca_scaled)
                
            elif method_name == "GMM":
                model = GaussianMixture(random_state=42, **params)
                labels = model.fit_predict(X_pca_scaled)
                
            elif method_name == "Spectral":
                model = SpectralClustering(random_state=42, affinity='nearest_neighbors', **params)
                labels = model.fit_predict(X_pca_scaled)
                
            elif method_name == "Birch":  
                model = Birch(**params)
                labels = model.fit_predict(X_pca_scaled)
            
            # evaluate clustering quality
            if len(np.unique(labels)) > 1:
                sil_score = silhouette_score(X_pca_scaled, labels)
                cal_score = calinski_harabasz_score(X_pca_scaled, labels)
                dav_score = davies_bouldin_score(X_pca_scaled, labels)
                
                print(f"   Silhouette Score: {sil_score:.3f}")
                print(f"   Calinski-Harabasz: {cal_score:.2f}")
                print(f"   Davies-Bouldin: {dav_score:.3f}")
                
                clustering_results[method_name] = {
                    'labels': labels,
                    'model': model,
                    'silhouette': sil_score,
                    'calinski': cal_score,
                    'davies_bouldin': dav_score
                }
                
                # Print cluster composition
                print(f"   Cluster composition:")
                for cluster_id in np.unique(labels):
                    cluster_teams = pca_df.index[labels == cluster_id].tolist()
                    print(f"      Cluster {cluster_id}: {cluster_teams}")
                    
        except Exception as e:
            print(f" Error with {method_name}: {e}")
    
    return clustering_results


def visualize_pca_clusters(pca_results, clustering_results):
    """
    Visualize clustering results on PCA space
    """
    print("\n CLUSTERING VISUALIZATION")
    print("-" * 50)

    #Extract PCA scores (teams and components) and variance ratios for labeling axes
    pca_df = pca_results['pca_df']
    explained_variance = pca_results['explained_variance_ratio']
    
    # create a visualization for each clustering method
    for method_name, result in clustering_results.items():
        labels = result['labels']
        sil_score = result['silhouette']
        
        # create plot dataframe
        plot_df = pca_df.copy()
        plot_df['Team'] = plot_df.index
        plot_df['Cluster'] = labels.astype(str)
        
        # 2D visualization for first two components
        if pca_df.shape[1] >= 2:
            fig = px.scatter(
                plot_df, x='Style_1', y='Style_2',
                color='Cluster', text='Team',
                category_orders={"Cluster": sorted(plot_df["Cluster"].unique(), key=lambda x: int(x))}, 
                title=f'{method_name} Clustering on Tactical Styles',
                labels={'Style_1': f'Style 1 ({explained_variance[0]:.1%})',
                       'Style_2': f'Style 2 ({explained_variance[1]:.1%})'},
                width=1000, height=600
            )
            fig.update_traces(textposition='top center')
            fig.show()
        
        # 3D visualization if we have at least 3 components
        if pca_df.shape[1] >= 3:
            fig = px.scatter_3d(
                plot_df, x='Style_1', y='Style_2', z='Style_3',
                color='Cluster', text='Team',
                category_orders={"Cluster": sorted(plot_df["Cluster"].unique(), key=lambda x: int(x))}, 
                title=f'{method_name} Clustering - 3D Tactical Space',
                labels={'Style_1': f'Style 1 ({explained_variance[0]:.1%})',
                       'Style_2': f'Style 2 ({explained_variance[1]:.1%})',
                       'Style_3': f'Style 3 ({explained_variance[2]:.1%})'},
                width=1000, height=700
            )
            fig.show()

def compute_global_discriminant_features(pca_results, clustering_results):
    """
    Compute average F-ratio (feature importance) across all clustering methods.
    """
    print("\n GLOBAL MOST DISCRIMINANT FEATURES (FOR ALL CLUSTERINGS)")
    print("=" * 60)

    # Prepare a dictionary to store F-ratios per feature for each clustering method
    original_data = pca_results['original_data']
    global_f_ratios = {feature: [] for feature in original_data.columns}

    # Loop over results from each clustering algorithm
    for method_name, result in clustering_results.items():
        labels = result['labels']
        original_with_clusters = original_data.copy()
        original_with_clusters['Cluster'] = labels
        unique_clusters = np.unique(labels)

        #Loop through every original metric (feature)
        for feature in original_data.columns:
            feature_values = original_with_clusters[feature].values
            overall_mean = np.mean(feature_values)

            between_ss = 0
            within_ss = 0

            # Loop through each cluster to accumulate between/within variance
            for cluster_id in unique_clusters:
                cluster_vals = original_with_clusters[original_with_clusters['Cluster'] == cluster_id][feature].values
                cluster_mean = np.mean(cluster_vals)
                between_ss += len(cluster_vals) * (cluster_mean - overall_mean) ** 2
                within_ss += np.sum((cluster_vals - cluster_mean) ** 2)

            # Compute the F-ratio: higher means the feature better separates clusters
            f_ratio = between_ss / within_ss if within_ss > 0 else 0
            # Save this F-ratio for the current feature and clustering method
            global_f_ratios[feature].append(f_ratio)

    # average F-ratios across algorithms
    avg_f_ratios = [(feature, np.mean(f_vals)) for feature, f_vals in global_f_ratios.items()]
    avg_f_ratios.sort(key=lambda x: x[1], reverse=True)

    print(" Averaged most discriminating features across clustering methods:")
    for i, (feature, avg_f) in enumerate(avg_f_ratios):
        print(f"   {i+1:2d}. {feature:<25} Avg F-ratio = {avg_f:.3f}")

    # main execution function
def run_pca_tactical_analysis(df_stocked):
    """
    Run a complete PCA-based tactical analysis
    """
    # 1: Apply PCA transformation
    pca_results = apply_pca_transformation(df_stocked, variance_threshold=0.85)
    
    # 2: Apply clustering to PCA components
    clustering_results = cluster_pca_styles(pca_results, max_clusters=9)
    
    # 3: Visualize clustering results
    visualize_pca_clusters(pca_results, clustering_results)

    # 4: Compute global discriminant features across all clusterings
    compute_global_discriminant_features(pca_results, clustering_results)

    #Return results for further analysis
    return {
        'pca_results': pca_results,
        'clustering_results': clustering_results
    }


# Run main function
results = run_pca_tactical_analysis(df_stocked)

#--------------------------------------------------------------------------------------------------------
#--------------------------------------------------------------------------------------------------------
#--------------------------------------------------------------------------------------------------------
# Additional functionalities if needed

# save the scores per team based on average with colors in images

# calculate average and categorize performance with 6 levels
metric_averages = df_stocked.mean()
performance = pd.DataFrame(index=df_stocked.index, columns=df_stocked.columns)

# Assign performance ratings for each team and metric using league average ± standard deviations
for col in df_stocked.columns:
    avg = metric_averages[col]
    std = df_stocked[col].std()
    for team in df_stocked.index:
        val = df_stocked.loc[team, col]
        if val >= avg + 2*std:
            cat = "+++"  # very good performance (2+ std above avg)
        elif val >= avg + std:
            cat = "++"   # good performance (1-2 std above avg)
        elif val >= avg:
            cat = "+"    # Above average performance
        elif val <= avg - 2*std:
            cat = "---"  # Very poor performance (2+ std below avg)
        elif val <= avg - std:
            cat = "--"   # Poor performance (1-2 std below avg)
        else:
            cat = "-"    # Below average performance
        performance.loc[team, col] = cat

# combine value and label
df_display = df_stocked.round(2).astype(str) + " (" + performance + ")"
df_display.loc["League Average"] = [f"{metric_averages[col]:.2f} (avg)" for col in df_stocked.columns]

# save Horizontal Table ----------
plt.figure(figsize=(len(df_display.columns) * 2.2, len(df_display) * 0.35))
sns.set(font_scale=0.72)
table = plt.table(cellText=df_display.values,
                  rowLabels=df_display.index,
                  colLabels=df_display.columns,
                  loc='center',
                  cellLoc='center')
table.auto_set_font_size(False)
table.set_fontsize(6.5)
table.scale(1.05, 1.08)

# add colors based on performance categories (6 levels)
for i in range(len(df_display)):
    for j in range(len(df_display.columns)):
        cell_text = df_display.iloc[i, j]
        if "(+++)" in cell_text:
            table[(i+1, j)].set_facecolor('#006400')  # Very dark green
            table[(i+1, j)].set_text_props(weight='bold', color='white')
        elif "(++)" in cell_text:
            table[(i+1, j)].set_facecolor('#2E8B57')  # Dark green
            table[(i+1, j)].set_text_props(weight='bold', color='white')
        elif "(+)" in cell_text:
            table[(i+1, j)].set_facecolor('#90EE90')  # Light green
            table[(i+1, j)].set_text_props(weight='bold')
        elif "(---)" in cell_text:
            table[(i+1, j)].set_facecolor('#8B0000')  # Very dark red
            table[(i+1, j)].set_text_props(weight='bold', color='white')
        elif "(--)" in cell_text:
            table[(i+1, j)].set_facecolor('#CD5C5C')  # Dark red
            table[(i+1, j)].set_text_props(weight='bold', color='white')
        elif "(-)" in cell_text:
            table[(i+1, j)].set_facecolor('#FFB6C1')  # Light red
            table[(i+1, j)].set_text_props(weight='bold')
        elif "(avg)" in cell_text:
            table[(i+1, j)].set_facecolor('#D3D3D3')  # Light gray
            table[(i+1, j)].set_text_props(weight='bold')

plt.title('Team Tactical Metrics - Horizontal View', fontsize=14, fontweight='bold', pad=20)
plt.axis('off')
plt.savefig("team_tactical_metrics_performance_table_PremierLeague_1516_VAEP+OTHER.png", bbox_inches='tight', dpi=300)


print("\nLegend:")
print("+++ (Very Dark Green): Exceptional - Above average + 2 standard deviations")
print("++ (Dark Green): Very Good - Above average + 1 standard deviation")
print("+ (Light Green): Good - Above average")
print("- (Light Red): Below average")
print("-- (Dark Red): Poor - Below average - 1 standard deviation")
print("--- (Very Dark Red): Very Poor - Below average - 2 standard deviations")
print("avg (Gray): League average")

# Display the categorized data 
print("\n Performance Categories:")
print(performance)

# Display average scores per metric
print("\n Average Scores per Metric:")
print(metric_averages.round(2))